## INSTALL PACKAGE

In [ ]:
%pip install catboost

In [ ]:
%pip install tensorflow

In [ ]:
%pip install --upgrade tensorflow

In [ ]:
%pip install shap

## IMPORT LIBRARIES

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.pylab as plt
import numpy as np
from scipy.interpolate import interp1d
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from math import sqrt
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.metrics import mean_absolute_error
import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Conv1D, Flatten
import pandas as pd
import numpy as np
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, LSTM
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import shap

## IMPORT DATASET

In [2]:
df = pd.read_csv('merged_data_v1.csv', encoding='unicode_escape', decimal=',', sep=';')

# CRISP-DM

## STEP 1: BUSINESS UNDERSTANDING

*skip this parts*

## STEP 2: DATA UNDERSTANDING

### Data Verifications

In [3]:
df = df.drop(columns=['Unnamed: 40'])

In [4]:
datetime_cols = df.loc[:, 'date':'time'].columns
input_cols = df.loc[:, 'ni_in':'t_tic163'].columns
output_cols = df.loc[:, 'furnace_temp':'loi_kalsin'].columns

# drop rows with on 4/1/2021 to 6/1/2021 and 21/1/2021 to 22/1/2021
df = df[~df['date'].between(44200, 44202)]
df = df[~df['date'].between(44217, 44218)]

shifted_output_df = df[output_cols].shift(16)
df = pd.concat([df[datetime_cols], df[input_cols], shifted_output_df], axis=1)
df = df.drop(index=range(16))

#shift column tco, current, load, reductor_consume, charge_kiln 7 rows up
columns_to_shift = ['tco', 'current', 'load', 'reductor_consume', 'charge_kiln']

# Shift kolom tertentu ke atas 7 baris
shifted_columns = df[columns_to_shift].shift(-7)

# Mengganti kolom asli dengan versi yang telah di-shift
df[columns_to_shift] = shifted_columns

# Menghapus baris terakhir sebanyak 7 baris karena menjadi NaN setelah shift
df = df.iloc[:-7]

df = df.reset_index(drop=True)
print(df.head())

    date      time  ni_in  fe_in  cao_in  al2o3_in   s_m    bc  loi_in  \
0  44197  0.666667   2.05  12.77    0.45      1.37  1.86  0.55   11.17   
1  44197  0.708333   2.00  10.97    0.36      1.13  1.86  0.54   12.98   
2  44197  0.750000   2.00  10.97    0.36      1.13  1.86  0.54   12.98   
3  44197  0.791667   2.00  10.97    0.36      1.13  1.86  0.54   12.98   
4  44197  0.833333   2.00  10.97    0.36      1.13  1.86  0.54   12.98   

   mc_kilnfeed  ...  ni_met  c_met  si_met   fe_met   s_met  ni_slag  fe_slag  \
0        20.43  ...    20.8  1.294   1.284  74.8144  0.5044     0.07     7.22   
1        22.20  ...    20.8  1.294   1.284  74.8144  0.5044     0.07     7.22   
2        22.20  ...    20.8  1.294   1.284  74.8144  0.5044     0.07     7.22   
3        22.20  ...    20.8  1.294   1.284  74.8144  0.5044     0.07     7.22   
4        22.20  ...    20.8  1.294   1.284  74.8144  0.5044     0.07     7.22   

   t_kalsin  pic_161  loi_kalsin  
0     627.0     -3.0        0.71 

In [5]:
df = df.drop(columns=['date', 'time'])
df.columns
df.head()

,ni_in,fe_in,cao_in,al2o3_in,s_m,bc,loi_in,mc_kilnfeed,fc_coal,gcv_coal,...,ni_met,c_met,si_met,fe_met,s_met,ni_slag,fe_slag,t_kalsin,pic_161,loi_kalsin
0,2.05,12.77,0.45,1.37,1.86,0.55,11.17,20.43,43.85,6698.12,...,20.8,1.294,1.284,74.8144,0.5044,0.07,7.22,627.0,-3.0,0.71
1,2.00,10.97,0.36,1.13,1.86,0.54,12.98,22.20,45.12,6920.98,...,20.8,1.294,1.284,74.8144,0.5044,0.07,7.22,607.0,-2.7,0.71
2,2.00,10.97,0.36,1.13,1.86,0.54,12.98,22.20,45.12,6920.98,...,20.8,1.294,1.284,74.8144,0.5044,0.07,7.22,614.0,-3.1,0.71
3,2.00,10.97,0.36,1.13,1.86,0.54,12.98,22.20,45.12,6920.98,...,20.8,1.294,1.284,74.8144,0.5044,0.07,7.22,620.0,-0.9,0.90
4,2.00,10.97,0.36,1.13,1.86,0.54,12.98,22.20,45.12,6920.98,...,20.8,1.294,1.284,74.8144,0.5044,0.07,7.22,610.0,-3.0,0.90


### EDA

In [6]:
df.shape

(601, 38)

In [7]:
df.dtypes

ni_in               float64
fe_in               float64
cao_in              float64
al2o3_in            float64
s_m                 float64
bc                  float64
loi_in              float64
mc_kilnfeed         float64
fc_coal             float64
gcv_coal            float64
tco                 float64
dry_ore             float64
current             float64
load                float64
realisasi_beban     float64
rpm                 float64
pry_p               float64
sec_p               float64
pry_v               float64
sec_v               float64
total_coal          float64
a_f_ratio           float64
kg_tco              float64
reductor_consume    float64
charge_kiln         float64
t_tic162            float64
t_tic163            float64
furnace_temp        float64
ni_met              float64
c_met               float64
si_met              float64
fe_met              float64
s_met               float64
ni_slag             float64
fe_slag             float64
t_kalsin            

In [8]:
df.head()

,ni_in,fe_in,cao_in,al2o3_in,s_m,bc,loi_in,mc_kilnfeed,fc_coal,gcv_coal,...,ni_met,c_met,si_met,fe_met,s_met,ni_slag,fe_slag,t_kalsin,pic_161,loi_kalsin
0,2.05,12.77,0.45,1.37,1.86,0.55,11.17,20.43,43.85,6698.12,...,20.8,1.294,1.284,74.8144,0.5044,0.07,7.22,627.0,-3.0,0.71
1,2.00,10.97,0.36,1.13,1.86,0.54,12.98,22.20,45.12,6920.98,...,20.8,1.294,1.284,74.8144,0.5044,0.07,7.22,607.0,-2.7,0.71
2,2.00,10.97,0.36,1.13,1.86,0.54,12.98,22.20,45.12,6920.98,...,20.8,1.294,1.284,74.8144,0.5044,0.07,7.22,614.0,-3.1,0.71
3,2.00,10.97,0.36,1.13,1.86,0.54,12.98,22.20,45.12,6920.98,...,20.8,1.294,1.284,74.8144,0.5044,0.07,7.22,620.0,-0.9,0.90
4,2.00,10.97,0.36,1.13,1.86,0.54,12.98,22.20,45.12,6920.98,...,20.8,1.294,1.284,74.8144,0.5044,0.07,7.22,610.0,-3.0,0.90


In [9]:
df.columns

Index(['ni_in', 'fe_in', 'cao_in', 'al2o3_in', 's_m', 'bc', 'loi_in',
       'mc_kilnfeed', 'fc_coal', 'gcv_coal', 'tco', 'dry_ore', 'current',
       'load', 'realisasi_beban', 'rpm', 'pry_p', 'sec_p', 'pry_v', 'sec_v',
       'total_coal', 'a_f_ratio', 'kg_tco', 'reductor_consume', 'charge_kiln',
       't_tic162', 't_tic163', 'furnace_temp', 'ni_met', 'c_met', 'si_met',
       'fe_met', 's_met', 'ni_slag', 'fe_slag', 't_kalsin', 'pic_161',
       'loi_kalsin'],
      dtype='object')

In [10]:
df.describe()

,ni_in,fe_in,cao_in,al2o3_in,s_m,bc,loi_in,mc_kilnfeed,fc_coal,gcv_coal,...,ni_met,c_met,si_met,fe_met,s_met,ni_slag,fe_slag,t_kalsin,pic_161,loi_kalsin
count,592.00000,592.000000,592.000000,592.000000,592.000000,592.000000,568.000000,593.000000,585.000000,585.000000,...,601.000000,601.000000,601.000000,601.000000,601.000000,601.000000,601.000000,513.000000,513.000000,575.000000
mean,2.04565,13.969198,0.543725,2.299029,1.794275,0.569865,11.017412,20.277361,43.807162,6617.216718,...,21.482622,1.187875,1.188554,74.405002,0.502005,0.063737,8.713244,613.097466,-3.279922,0.265704
std,0.06920,1.005655,0.110025,0.655441,0.114128,0.033383,1.130846,1.605423,2.595439,842.003193,...,0.570276,0.086174,0.099359,0.523199,0.051887,0.005039,1.191834,26.684175,0.761017,0.296641
min,1.71500,10.970000,0.290000,0.000000,1.555143,0.467150,9.580000,12.505000,34.000000,658.970000,...,17.990000,1.030000,0.972500,73.395000,0.416200,0.050000,4.990000,492.000000,-5.600000,0.000000
25%,2.00500,13.220000,0.460000,1.760000,1.723282,0.550000,10.302500,19.640000,42.730000,6587.010000,...,21.243333,1.145000,1.131667,73.972000,0.462167,0.060000,8.020000,599.000000,-3.800000,0.000000
50%,2.04000,13.990000,0.560000,2.475000,1.776706,0.573375,10.740000,20.605000,44.180000,6706.820000,...,21.540000,1.188333,1.180000,74.524200,0.487625,0.064000,8.723333,611.000000,-3.300000,0.160000
75%,2.09000,14.775000,0.620000,2.885000,1.840000,0.590000,11.440000,21.360000,45.120000,6920.980000,...,21.860000,1.256250,1.238750,74.673000,0.528500,0.066667,9.784000,623.000000,-2.800000,0.510000
max,2.19000,15.930000,0.805000,3.140000,2.213630,0.654692,16.410000,23.325000,49.050000,8485.760000,...,22.436667,1.483000,1.757000,76.737000,0.638000,0.072000,10.174000,754.000000,1.700000,1.040000


### Outlier Detection

In [ ]:
for column in df.columns:
    plt.figure(figsize=(10, 4))
    sns.boxplot(x=df[column])
    plt.title(f'Boxplot of {column}')
    plt.show()

In [ ]:
from scipy.stats import zscore

# Calculate Z-scores
z_scores = df.apply(zscore)
# Flag outliers (absolute Z-score > 3)
outliers_zscore = (z_scores.abs() > 3)

# Count outliers for each column
outlier_count = outliers_zscore.sum()
print(outlier_count)

In [ ]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1

# Define outlier bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Detect outliers
outliers_iqr = (df < lower_bound) | (df > upper_bound)

# Count outliers for each column
outlier_count_iqr = outliers_iqr.sum()
print(outlier_count_iqr)


### Data Visualization

#### All Variables

In [ ]:
df_input_only = df.drop(columns=output_cols)

# Plot histograms for input columns
df_input_only.hist(figsize=(12, 10), bins=20, color='skyblue', edgecolor='black')

# Show the plots
plt.tight_layout()
plt.show()

In [ ]:
df_output_only = df.drop(columns=input_cols)

# Plot histograms for input columns
df_output_only.hist(figsize=(12, 10), bins=20, color='skyblue', edgecolor='black')

# Show the plots
plt.tight_layout()
plt.show()

In [ ]:
input_data = df[input_cols]
input_data.boxplot(figsize=(40,12))

In [ ]:
corr_matrix_n = df.corr()
plt.figure(figsize=(24, 18))
sns.heatmap(corr_matrix_n, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix')
plt.show()

In [ ]:
# Flatten the correlation matrix and filter correlations
corr_pairs = corr_matrix_n.unstack()

# Sort the correlations from highest to lowest
sorted_corr = corr_pairs.sort_values(key=abs, ascending=False)

# Exclude self-correlations (correlation of a variable with itself)
filtered_corr = sorted_corr[sorted_corr.index.get_level_values(0) != sorted_corr.index.get_level_values(1)]

# Split into positive and negative correlations
positive_corr = filtered_corr[filtered_corr > 0]
negative_corr = filtered_corr[filtered_corr < 0]

# Display top correlations
print("Top Positive Correlations:\n", positive_corr.head(50))
print("\nTop Negative Correlations:\n", negative_corr.head(50))

In [ ]:
# Combined Plotting Function for a Single Variable vs Multiple Inputs
def plot_single_vs_inputs_combined(variable, input_cols, df, title_prefix=""):
    num_inputs = len(input_cols)  # Total number of input columns
    cols = 6  # Number of columns for subplots (adjustable)
    rows = -(-num_inputs // cols)  # Calculate rows (ceiling division)

    fig, axes = plt.subplots(rows, cols, figsize=(5 * cols, 5 * rows))  # Adjust figure size
    axes = axes.flatten()  # Flatten axes for easy iteration

    for i, input_col in enumerate(input_cols):
        sns.scatterplot(x=df[input_col], y=df[variable], ax=axes[i])
        axes[i].set_title(f"{title_prefix}{variable} vs {input_col}")
        axes[i].set_xlabel(input_col)
        axes[i].set_ylabel(variable)
        axes[i].grid(True)

    # Hide any unused subplots
    for i in range(len(input_cols), len(axes)):
        fig.delaxes(axes[i])

    plt.tight_layout()
    plt.show()

# Reductor Consume vs RK Inputs Combined
plot_single_vs_inputs_combined('reductor_consume', input_cols, df, title_prefix="RK: ")

In [ ]:
# Total Coal vs RK Inputs Combined
plot_single_vs_inputs_combined('total_coal', input_cols, df, title_prefix="RK: ")

In [ ]:
# Combined Scatter Plot Function
def plot_scatter_vs_inputs(variable, input_cols, df, cols=3, title_prefix=""):
    """
    Generate scatter plots for a single output variable vs multiple input variables.

    Parameters:
    - variable (str): The output variable to plot on the Y-axis.
    - input_cols (list): List of input variable names for the X-axis.
    - df (DataFrame): The dataframe containing the data.
    - cols (int): Number of columns in the subplot grid.
    - title_prefix (str): Prefix to add to subplot titles.
    """
    num_inputs = len(input_cols)  # Total number of input columns
    rows = -(-num_inputs // cols)  # Calculate number of rows with ceiling division

    fig, axes = plt.subplots(rows, cols, figsize=(5 * cols, 5 * rows))  # Adjust figure size
    axes = axes.flatten()  # Flatten axes for easy iteration

    # Loop through each input column and create scatter plots
    for i, input_col in enumerate(input_cols):
        sns.scatterplot(x=df[input_col], y=df[variable], ax=axes[i], color='blue')
        axes[i].set_title(f"{title_prefix}{variable} vs {input_col}")
        axes[i].set_xlabel(input_col)
        axes[i].set_ylabel(variable)
        axes[i].grid(True)

    # Hide unused subplots if any
    for i in range(len(input_cols), len(axes)):
        fig.delaxes(axes[i])

    plt.tight_layout()  # Adjust spacing between subplots
    plt.show()

# Example Use Case:
# Reductor Consume vs RK Inputs Combined
plot_scatter_vs_inputs('reductor_consume', input_cols, df, cols=6, title_prefix="RK: ")

In [ ]:
# Total Coal vs RK Inputs Combined
plot_scatter_vs_inputs('total_coal', input_cols, df, cols=6, title_prefix="RK: ")

In [ ]:
x_columns = df.loc[:, 'reductor_consume':'charge_kiln'].columns
y_columns = df.loc[:, 'ni_met':'fe_slag'].columns

# Setup grid untuk subplots
n_rows = len(x_columns)
n_cols = len(y_columns)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 8), sharex=False, sharey=False)

# Loop untuk membuat scatter plot setiap pasangan kolom
for i, x_col in enumerate(x_columns):
    for j, y_col in enumerate(y_columns):
        ax = axes[i, j]
        ax.scatter(df[x_col], df[y_col], alpha=0.7, color='blue', edgecolors='black')
        ax.set_title(f'{x_col} vs {y_col}', fontsize=8)
        ax.tick_params(axis='both', which='major', labelsize=6)

# Menyesuaikan tata letak agar tidak tumpang tindih
plt.tight_layout()
plt.savefig('reductors-compositions_plot.png', dpi=300)
plt.show()

In [ ]:
# Define x_columns and y_columns
x_columns = ['total_coal']  # Explicitly list the column name
y_columns = df.loc[:, 'ni_met':'fe_slag'].columns  # Slice of column names for y-axis

# Setup grid for subplots
n_rows = len(x_columns)
n_cols = len(y_columns)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 8), sharex=False, sharey=False)

# Handle the case when there's only 1 row or column
if n_rows == 1 and n_cols == 1:
    axes = [[axes]]  # Make it 2D
elif n_rows == 1:
    axes = [axes]  # Wrap into a list
elif n_cols == 1:
    axes = [[ax] for ax in axes]  # Transpose axes to 2D list

# Loop to create scatter plots
for i, x_col in enumerate(x_columns):
    for j, y_col in enumerate(y_columns):
        ax = axes[i][j]
        ax.scatter(df[x_col], df[y_col], alpha=0.7, color='blue', edgecolors='black')
        ax.set_title(f'{x_col} vs {y_col}', fontsize=8)
        ax.tick_params(axis='both', which='major', labelsize=6)

# Adjust layout to avoid overlap
plt.tight_layout()
plt.show()

#### Rotary Kiln Variables

In [25]:
rk_in_cols = [
    'ni_in', 'fe_in', 'cao_in', 'al2o3_in', 's_m', 'bc',
    'loi_in', 'mc_kilnfeed', 'fc_coal', 'gcv_coal', 'tco', 'dry_ore',
    'rpm', 'pry_p', 'sec_p', 'pry_v', 'sec_v', 'total_coal', 'a_f_ratio', 'kg_tco', 'reductor_consume', 'charge_kiln'
]

rk_out_cols = [
    't_tic162', 't_tic163','t_kalsin', 'loi_kalsin', 'pic_161'
]

In [ ]:
# Compute correlation matrix for RK inputs and outputs
corr_matrix_rk = df[rk_in_cols + rk_out_cols].corr()

# Plot the heatmap
plt.figure(figsize=(18, 12))
sns.heatmap(corr_matrix_rk, annot=True, fmt=".2f", cmap="coolwarm", cbar=True)
plt.title("Correlation Heatmap of RK Inputs and Outputs")
plt.show()


In [ ]:
df[rk_in_cols].hist(figsize=(12, 10), bins=20, color='skyblue', edgecolor='black')
plt.suptitle("Histograms of RK Input Columns", y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
df[rk_out_cols].hist(figsize=(8, 6), bins=20, color='orange', edgecolor='black')
plt.suptitle("Histograms of RK Output Columns", y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
input_data_rk = df[rk_in_cols]
input_data_rk.boxplot(figsize=(40,12))

In [ ]:
output_data_rk = df[rk_out_cols]
output_data_rk.boxplot(figsize=(40,12))

In [ ]:
def plot_relationships(input_cols, output_cols, df, title_prefix):
    for output_col in output_cols:
        for input_col in input_cols:
            plt.figure(figsize=(8, 6))
            sns.scatterplot(x=df[input_col], y=df[output_col])
            plt.title(f"{title_prefix}: {input_col} vs {output_col}")
            plt.xlabel(input_col)
            plt.ylabel(output_col)
            plt.grid(True)
            plt.show()

# Input RK vs Output RK
plot_relationships(rk_in_cols, rk_out_cols, df, "RK Input vs RK Output")

#### Electric Furnace Variables

In [32]:
ef_in_cols = [
    'ni_in', 'fe_in', 'cao_in', 'al2o3_in', 's_m', 'bc', 'dry_ore',
    'fc_coal', 'gcv_coal', 'tco', 'current', 'load', 'realisasi_beban',
    'total_coal', 'charge_kiln', 't_kalsin', 'loi_kalsin', 'pic_161'
]

ef_out_cols = [
    't_tic162', 't_tic163', 'furnace_temp',
    'ni_met', 'c_met', 'si_met', 'fe_met', 's_met', 'ni_slag',
    'fe_slag'
]

In [ ]:
# Compute correlation matrix for EF inputs and outputs
corr_matrix_ef = df[ef_in_cols + ef_out_cols].corr()

# Plot the heatmap
plt.figure(figsize=(18, 12))
sns.heatmap(corr_matrix_ef, annot=True, fmt=".2f", cmap="coolwarm", cbar=True)
plt.title("Correlation Heatmap of EF Inputs and Outputs")
plt.show()


In [ ]:
df[ef_in_cols].hist(figsize=(12, 10), bins=20, color='skyblue', edgecolor='black')
plt.suptitle("Histograms of EF Input Columns", y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
df[ef_out_cols].hist(figsize=(8, 6), bins=20, color='orange', edgecolor='black')
plt.suptitle("Histograms of EF Output Columns", y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
input_data_ef = df[ef_in_cols]
input_data_ef.boxplot(figsize=(40,12))

In [ ]:
output_data_ef = df[ef_out_cols]
output_data_ef.boxplot(figsize=(40,12))

In [ ]:
# Input EF vs Output EF
plot_relationships(ef_in_cols, ef_out_cols, df, "EF Input vs EF Output")

# Plotting Reductor Consume vs All RK Input Variables
def plot_single_vs_inputs(variable, input_cols, df):
    for input_col in input_cols:
        plt.figure(figsize=(8, 6))
        sns.scatterplot(x=df[input_col], y=df[variable])
        plt.title(f"{variable} vs {input_col}")
        plt.xlabel(input_col)
        plt.ylabel(variable)
        plt.grid(True)
        plt.show()

## STEP 3: DATA PREPARATION

### Interpolation

In [11]:
df[input_cols.union(output_cols)] = df[input_cols.union(output_cols)].interpolate(method='linear', axis=0)
print(df[input_cols.union(output_cols)].isnull().sum())

a_f_ratio           0
al2o3_in            0
bc                  0
c_met               0
cao_in              0
charge_kiln         0
current             0
dry_ore             0
fc_coal             0
fe_in               0
fe_met              0
fe_slag             0
furnace_temp        0
gcv_coal            0
kg_tco              0
load                0
loi_in              0
loi_kalsin          0
mc_kilnfeed         0
ni_in               0
ni_met              0
ni_slag             0
pic_161             0
pry_p               0
pry_v               0
realisasi_beban     0
reductor_consume    0
rpm                 0
s_m                 0
s_met               0
sec_p               0
sec_v               0
si_met              0
t_kalsin            0
t_tic162            0
t_tic163            0
tco                 0
total_coal          0
dtype: int64


In [12]:
df.isnull().sum()

ni_in               0
fe_in               0
cao_in              0
al2o3_in            0
s_m                 0
bc                  0
loi_in              0
mc_kilnfeed         0
fc_coal             0
gcv_coal            0
tco                 0
dry_ore             0
current             0
load                0
realisasi_beban     0
rpm                 0
pry_p               0
sec_p               0
pry_v               0
sec_v               0
total_coal          0
a_f_ratio           0
kg_tco              0
reductor_consume    0
charge_kiln         0
t_tic162            0
t_tic163            0
furnace_temp        0
ni_met              0
c_met               0
si_met              0
fe_met              0
s_met               0
ni_slag             0
fe_slag             0
t_kalsin            0
pic_161             0
loi_kalsin          0
dtype: int64

### Normalization

In [13]:
scaler = MinMaxScaler()

# Fit and transform the DataFrame
normalized_data = scaler.fit_transform(df)

# Convert back to a DataFrame with the original column names
normalized_df = pd.DataFrame(normalized_data, columns=df.columns)

# Display the first few rows of the normalized DataFrame
print(normalized_df.head())

      ni_in     fe_in    cao_in  al2o3_in       s_m        bc    loi_in  \
0  0.705263  0.362903  0.310680  0.436306  0.462966  0.441768  0.232796   
1  0.600000  0.000000  0.135922  0.359873  0.462966  0.388446  0.497804   
2  0.600000  0.000000  0.135922  0.359873  0.462966  0.388446  0.497804   
3  0.600000  0.000000  0.135922  0.359873  0.462966  0.388446  0.497804   
4  0.600000  0.000000  0.135922  0.359873  0.462966  0.388446  0.497804   

   mc_kilnfeed   fc_coal  gcv_coal  ...    ni_met     c_met    si_met  \
0     0.732440  0.654485  0.771600  ...  0.631934  0.582781  0.397068   
1     0.896026  0.738870  0.800074  ...  0.631934  0.582781  0.397068   
2     0.896026  0.738870  0.800074  ...  0.631934  0.582781  0.397068   
3     0.896026  0.738870  0.800074  ...  0.631934  0.582781  0.397068   
4     0.896026  0.738870  0.800074  ...  0.631934  0.582781  0.397068   

     fe_met     s_met   ni_slag  fe_slag  t_kalsin   pic_161  loi_kalsin  
0  0.424716  0.397656  0.909091  0.

### Data Describing

In [14]:
pd.set_option('display.max_columns', None)

In [15]:
print(normalized_df.describe())

            ni_in       fe_in      cao_in    al2o3_in         s_m          bc  \
count  601.000000  601.000000  601.000000  601.000000  601.000000  601.000000   
mean     0.693975    0.608409    0.493676    0.735136    0.363335    0.547434   
std      0.145995    0.203810    0.212285    0.208865    0.172122    0.176780   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%      0.610526    0.453629    0.330097    0.560510    0.255341    0.441768   
50%      0.684211    0.616935    0.524272    0.800955    0.341117    0.566406   
75%      0.789474    0.786290    0.640777    0.925159    0.432593    0.655053   
max      1.000000    1.000000    1.000000    1.000000    1.000000    1.000000   

           loi_in  mc_kilnfeed     fc_coal    gcv_coal         tco  \
count  601.000000   601.000000  601.000000  601.000000  601.000000   
mean     0.215048     0.718641    0.649556    0.762006    0.282057   
std      0.163948     0.147408    0.170784    0.106254    0.

### Data Visualization After Normalization

In [ ]:
normalized_corr = normalized_df.corr()
plt.figure(figsize=(24, 18))
sns.heatmap(normalized_corr, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Normalized Correlation Matrix')
plt.show()

In [ ]:
input_data_normalized = normalized_df[input_cols]
input_data_normalized.boxplot(figsize=(40,12))

In [ ]:
output_data_normalized = normalized_df[output_cols]
output_data_normalized.boxplot(figsize=(40,12))

### Plotting with Compare Variables

In [ ]:
x_columns = normalized_df.loc[:, 'ni_in':'al2o3_in'].columns
y_columns = normalized_df.loc[:, 'ni_met':'fe_slag'].columns

# Setup grid untuk subplots
n_rows = len(x_columns)
n_cols = len(y_columns)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 20), sharex=False, sharey=False)

# Loop untuk membuat scatter plot setiap pasangan kolom
for i, x_col in enumerate(x_columns):
    for j, y_col in enumerate(y_columns):
        ax = axes[i, j]
        ax.scatter(normalized_df[x_col], normalized_df[y_col], alpha=0.7, color='blue', edgecolors='black')
        ax.set_title(f'{x_col} vs {y_col}', fontsize=8)
        ax.tick_params(axis='both', which='major', labelsize=6)

# Menyesuaikan tata letak agar tidak tumpang tindih
plt.tight_layout()
plt.savefig('compositions_plot.png', dpi=300)
plt.show()

In [ ]:
x_columns = normalized_df.loc[:, 'reductor_consume':'charge_kiln'].columns
y_columns = normalized_df.loc[:, 'ni_met':'fe_slag'].columns

# Setup grid untuk subplots
n_rows = len(x_columns)
n_cols = len(y_columns)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 8), sharex=False, sharey=False)

# Loop untuk membuat scatter plot setiap pasangan kolom
for i, x_col in enumerate(x_columns):
    for j, y_col in enumerate(y_columns):
        ax = axes[i, j]
        ax.scatter(normalized_df[x_col], normalized_df[y_col], alpha=0.7, color='blue', edgecolors='black')
        ax.set_title(f'{x_col} vs {y_col}', fontsize=8)
        ax.tick_params(axis='both', which='major', labelsize=6)

# Menyesuaikan tata letak agar tidak tumpang tindih
plt.tight_layout()
plt.savefig('reductors-compositions_plot.png', dpi=300)
plt.show()

In [ ]:
x_columns = ['rpm', 'pry_p', 'sec_p', 'pry_v', 'sec_v', 'total_coal', 'a_f_ratio', 'charge_kiln']
y_columns = ['t_kalsin', 'loi_kalsin', 'pic_161']

# Setup grid untuk subplots
n_rows = len(x_columns)
n_cols = len(y_columns)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 20), sharex=False, sharey=False)

# Loop untuk membuat scatter plot setiap pasangan kolom
for i, x_col in enumerate(x_columns):
    for j, y_col in enumerate(y_columns):
        ax = axes[i, j]
        ax.scatter(normalized_df[x_col], normalized_df[y_col], alpha=0.7, color='blue', edgecolors='black')
        ax.set_title(f'{x_col} vs {y_col}', fontsize=8)
        ax.tick_params(axis='both', which='major', labelsize=6)

# Menyesuaikan tata letak agar tidak tumpang tindih
plt.tight_layout()
plt.savefig('params-kalsin_plot.png', dpi=300)
plt.show()

In [ ]:
# Define x_columns as a list of columns
x_columns = normalized_df.loc[:, 'current':'load'].columns  # Select columns from 'voltage' to 'load'
y_column = ['furnace_temp']  # List of columns for y axis

# Setup grid for subplots
n_cols = len(x_columns)

# Create subplots (1 row, multiple columns)
fig, axes = plt.subplots(1, n_cols, figsize=(20, 4), sharex=False, sharey=False)

# Loop to create scatter plot for each pair of columns
for i, x_col in enumerate(x_columns):
    ax = axes[i]  # Corrected indexing
    ax.scatter(normalized_df[x_col], normalized_df[y_column[0]], alpha=0.7, color='blue', edgecolors='black')
    ax.set_title(f'{x_col} vs {y_column[0]}', fontsize=8)
    ax.tick_params(axis='both', which='major', labelsize=6)

# Adjust layout so that plots do not overlap
plt.tight_layout()
plt.show()

### PCA Analysis

In [ ]:
pca = PCA()
pca.fit(normalized_df)

# Menentukan jumlah komponen untuk mencapai 95% variansi kumulatif
cumulative_variance = pca.explained_variance_ratio_.cumsum()
n_components = next(i for i, total_var in enumerate(cumulative_variance) if total_var >= 0.99) + 1

# Melakukan PCA dengan jumlah komponen yang ditemukan
pca = PCA(n_components=n_components)
pca_data = pca.fit_transform(normalized_df)

# Membuat dataframe dengan hasil PCA
pca_df = pd.DataFrame(pca_data, columns=[f'PC{i+1}' for i in range(n_components)])

# Menampilkan hasil PCA
print(f'Number of principal components selected: {n_components}')
print(pca_df.head())

In [ ]:
# Plot Cumulative Explained Variance
plt.figure(figsize=(8, 6))
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o', linestyle='--', color='b')
plt.axhline(y=0.99, color='r', linestyle='-', label='99% Cumulative Variance')
plt.axvline(x=n_components, color='g', linestyle='--', label=f'Optimal Components = {n_components}')
plt.title('Cumulative Explained Variance by Principal Components', fontsize=14)
plt.xlabel('Number of Principal Components', fontsize=12)
plt.ylabel('Cumulative Explained Variance', fontsize=12)
plt.legend(loc='best')
plt.grid(True)
plt.show()

### Splitting Dataset

In [18]:
# X_merged = pd.concat([normalized_df[input_cols], pca_df], axis=1)
# y = normalized_df[output_cols]

# X_train_merged, X_test_merged, y_train, y_test = train_test_split(X_merged, y, test_size=0.2, random_state=42)

In [19]:
# inverted_actual_df = pd.concat([X_test_merged[input_cols], y_test], axis=1)
# inverted_actual_array = scaler.inverse_transform(inverted_actual_df)
# inverted_actual_df = pd.DataFrame(inverted_actual_array, columns=inverted_actual_df.columns)

In [20]:
# X_train = X_train_merged.drop(columns=normalized_df[input_cols], axis=1)
# X_test = X_test_merged.drop(columns=normalized_df[input_cols], axis=1)

In [16]:
# Use only the original input features from the normalized_df
X_merged = normalized_df[input_cols]
y = normalized_df[output_cols]

# Split the data into training and test sets
X_train_merged, X_test_merged, y_train, y_test = train_test_split(X_merged, y, test_size=0.2, random_state=42)

# Invert the scaling on the test data (combine X_test and y_test to inverse transform)
inverted_actual_df = pd.concat([X_test_merged[input_cols], y_test], axis=1)
inverted_actual_array = scaler.inverse_transform(inverted_actual_df)
inverted_actual_df = pd.DataFrame(inverted_actual_array, columns=inverted_actual_df.columns)

# Directly use X_train_merged and X_test_merged without dropping columns
X_train = X_train_merged
X_test = X_test_merged

## STEP 4: MODELLING

### RF

#### Hyperparameter Tuning

In [57]:
param_grid = {
    'n_estimators': [100, 200, 300],         # Number of trees in the forest
    'max_depth': [None, 10, 20, 30],        # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],        # Minimum samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],          # Minimum samples required to be at a leaf node
    'max_features': ['auto', 'sqrt', 'log2']  # Number of features to consider at each split
}


In [ ]:
grid_search = GridSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_grid=param_grid,
    cv=5,  # Cross-validation folds
    scoring='neg_mean_squared_error',  # Use MSE as the evaluation metric
    verbose=2,  # Display the progress of the search
    n_jobs=-1   # Use all CPU cores
)

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
# Best hyperparameters
print("Best Parameters for RF:", grid_search.best_params_)

# Best estimator
best_model_rf = grid_search.best_estimator_

# Make predictions with the best model
y_pred_rf_tuned = best_model_rf.predict(X_test)

#### Last Model

In [61]:
rf_model = RandomForestRegressor(n_estimators=100, max_depth=None, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

#### Best Model

In [62]:
# Initialize the Random Forest model with best parameters
rf_model = RandomForestRegressor(
    n_estimators=300,         # Number of trees
    max_depth=20,           # No limit on tree depth
    max_features='sqrt',      # Number of features to consider at each split
    min_samples_leaf=1,       # Minimum samples required in a leaf
    min_samples_split=2,      # Minimum samples required to split an internal node
    random_state=42           # For reproducibility
)

# Train the model
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_rf = rf_model.predict(X_test)

### MLP

#### Hyperparameter Tuning

In [63]:
param_grid_mlp = {
    'hidden_layer_sizes': [(64, 32), (128, 64), (128, 64, 32)],  # Vary the number of layers and nodes
    'activation': ['relu', 'tanh'],                             # Activation functions
    'solver': ['adam', 'sgd'],                                  # Optimizers
    'alpha': [0.0001, 0.001, 0.01],                             # Regularization term
    'learning_rate_init': [0.01, 0.02, 0.05],                   # Initial learning rates
    'max_iter': [200, 300, 500]                                 # Maximum number of iterations
}


In [64]:
# Initialize GridSearchCV
grid_search_mlp = GridSearchCV(
    estimator=MLPRegressor(random_state=42, verbose=True),
    param_grid=param_grid_mlp,
    scoring='neg_mean_squared_error',  # Scoring metric
    cv=5,                              # 5-fold cross-validation
    verbose=2,                         # Detailed output during the search
    n_jobs=-1                          # Use all CPU cores
)

In [ ]:
# Fit GridSearchCV
grid_search_mlp.fit(X_train, y_train)

In [ ]:
# Best parameters
print("Best Parameters for MLP:", grid_search_mlp.best_params_)

# Best model
best_model_mlp = grid_search_mlp.best_estimator_

# Make predictions using the best model
y_pred_mlp_tuned = best_model_mlp.predict(X_test)

#### Last Model

In [ ]:
mlp = MLPRegressor(hidden_layer_sizes=(64, 32),  # Example hidden layers
                   max_iter=500,
                   random_state=42,
                   learning_rate_init=0.02,
                   verbose=True)

# Fit the model
mlp.fit(X_train, y_train)

# Predict on test set
y_pred_mlp = mlp.predict(X_test)

#### Best Model

In [ ]:
# Initialize the MLPRegressor with the best parameters
mlp = MLPRegressor(
    hidden_layer_sizes=(128, 64),  # Optimized hidden layers
    activation='relu',             # Activation function
    solver='adam',                 # Optimized solver
    alpha=0.01,                    # Regularization term
    learning_rate_init=0.01,       # Initial learning rate
    max_iter=200,                  # Maximum number of iterations
    random_state=42,               # For reproducibility
    verbose=True                   # Output training progress
)

# Train the model
mlp.fit(X_train, y_train)

# Make predictions on the test set
y_pred_mlp = mlp.predict(X_test)

### SVR

#### Hyperparameter Tuning

In [69]:
# Define parameter grid for SVR
param_grid_svr = {
    'estimator__kernel': ['rbf', 'linear', 'poly', 'sigmoid'],  # Include all kernel types
    'estimator__C': [0.1, 1, 10, 100],                        # Regularization parameter
    'estimator__epsilon': [0.01, 0.1, 1],                     # Tolerance for regression loss
    'estimator__gamma': ['scale', 'auto'],                    # Kernel coefficient for RBF, poly, sigmoid
    'estimator__degree': [2, 3, 4],                           # Degree for polynomial kernel
    'estimator__coef0': [0.0, 0.1, 0.5]                      # Coef0 for poly and sigmoid kernels
}

In [70]:
# Wrap SVR with MultiOutputRegressor
multi_svr_model = MultiOutputRegressor(SVR())

In [71]:
# Initialize GridSearchCV
grid_search_svr = GridSearchCV(
    estimator=multi_svr_model,  # Model wrapped in MultiOutputRegressor
    param_grid=param_grid_svr, # Parameter grid
    cv=5,                      # 3-fold cross-validation
    scoring='neg_mean_squared_error', # Scoring metric
    verbose=2,                 # Detailed output during the search
    n_jobs=-1                  # Use all available CPU cores
)

In [ ]:
# Fit GridSearchCV
grid_search_svr.fit(X_train, y_train)

In [ ]:
# Best parameters
print("Best Parameters for SVR:", grid_search_svr.best_params_)

# Best model
best_model_svr = grid_search_svr.best_estimator_

# Make predictions using the best model
y_pred_svr_tuned = best_model_svr.predict(X_test)

##### Last Model

In [74]:
# Initialize SVR
svr = SVR(kernel='rbf', C=10.0, epsilon=0.1, gamma='auto')

# Wrap SVR with MultiOutputRegressor
multi_output_svr = MultiOutputRegressor(svr)

# Train the model
multi_output_svr.fit(X_train, y_train)

# Predict on test set
y_pred_svr = multi_output_svr.predict(X_test)

#### Best Model

In [75]:
# Initialize SVR with the best parameters
svr = SVR(
    kernel='rbf',        # Kernel type
    C=10,                # Regularization parameter (Best parameter: 10)
    epsilon=0.01,        # Epsilon in the loss function (Best parameter: 0.01)
    gamma='scale',       # Kernel coefficient (Best parameter: 'scale')
    degree=2,            # Degree of the polynomial kernel (Best parameter: 2, only for 'poly')
    coef0=0.0            # Independent term in kernel function (Best parameter: 0.0, only for 'poly' and 'sigmoid')
)

# Wrap SVR with MultiOutputRegressor for handling multi-output regression
multi_output_svr = MultiOutputRegressor(svr)

# Train the model
multi_output_svr.fit(X_train, y_train)

# Predict on the test set
y_pred_svr = multi_output_svr.predict(X_test)

### RR

#### Hyperparameter Tuning

In [76]:
# Define parameter grid for Ridge Regression
param_grid_ridge = {
    'alpha': [0.001, 0.01, 0.1, 1, 10, 100],  # Regularization strength
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga', 'lbfgs']  # Different solvers
}

# Initialize Ridge Regression model
ridge_model = Ridge()

# Initialize GridSearchCV
grid_search_ridge = GridSearchCV(
    estimator=ridge_model,
    param_grid=param_grid_ridge,
    scoring='neg_mean_squared_error',  # Use MSE as the scoring metric
    cv=5,                              # 5-fold cross-validation
    verbose=2,                         # Print detailed progress
    n_jobs=-1                          # Use all available CPU cores
)

In [ ]:
# Fit GridSearchCV
grid_search_ridge.fit(X_train, y_train)

In [ ]:
# Best parameters
print("Best Parameters for RR:", grid_search_ridge.best_params_)

# Best model
best_model_rr = grid_search_ridge.best_estimator_

# Make predictions using the best model
y_pred_rr_tuned = best_model_rr.predict(X_test)

#### Last Model

In [79]:
# Initialize Ridge Regression model
ridge_model = Ridge(alpha=0.1, solver='svd')  # Regularization strength

# Train the model
ridge_model.fit(X_train, y_train)

# Predict on test set
y_pred_rr = ridge_model.predict(X_test)

#### Best Model

In [17]:
# Initialize Ridge Regression model with the best parameters
ridge_model = Ridge(
    alpha=0.1,         # Regularization strength
    solver='sparse_cg'  # Solver for optimization
)

# Train the model
ridge_model.fit(X_train.to_numpy(), y_train.to_numpy())

# Predict on the test set
y_pred_rr = ridge_model.predict(X_test.to_numpy())

### LASSO

#### Hyperparameter Tuning

In [81]:
# Define parameter grid for Lasso Regression
param_grid_lasso = {
    'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10],  # Regularization strength
    'max_iter': [1000, 5000, 10000],            # Maximum number of iterations for convergence
    'selection': ['cyclic', 'random']           # Coordinate descent strategy
}

# Initialize Lasso Regression model
lasso_model = Lasso()

# Initialize GridSearchCV
grid_search_lasso = GridSearchCV(
    estimator=lasso_model,
    param_grid=param_grid_lasso,
    scoring='neg_mean_squared_error',  # Use MSE as the scoring metric
    cv=5,                              # 5-fold cross-validation
    verbose=2,                         # Detailed progress output
    n_jobs=-1                          # Use all CPU cores
)

In [ ]:
# Fit GridSearchCV
grid_search_lasso.fit(X_train, y_train)

In [ ]:
# Best parameters
print("Best Parameters for LASSO:", grid_search_lasso.best_params_)

# Best model
best_model_lasso = grid_search_lasso.best_estimator_

# Make predictions using the best model
y_pred_lasso_tuned = best_model_lasso.predict(X_test)

#### Last Model

In [84]:
lasso = Lasso(alpha=0.01)
lasso.fit(X_train, y_train)
y_pred_lasso = lasso.predict(X_test)

#### Best Model

In [ ]:
# Initialize Lasso Regression model with the best parameters
lasso = Lasso(
    alpha=0.0001,       # Regularization strength
    max_iter=1000,      # Maximum number of iterations
    selection='random'  # Random coordinate descent for optimization
)

# Train the model
lasso.fit(X_train, y_train)

# Predict on the test set
y_pred_lasso = lasso.predict(X_test)

### kNN

#### Hyperparameter Tuning

In [86]:
# Define parameter grid for KNN Regressor
param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9, 11],  # Number of neighbors
    'weights': ['uniform', 'distance'],  # Weight function used in prediction
    'metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski'],  # Distance metric
    'leaf_size': [20, 30, 40],  # Leaf size for KDTree or BallTree algorithm
    'p': [1, 2]  # Power parameter for the Minkowski distance
}

# Initialize KNeighborsRegressor model
knn_model = KNeighborsRegressor()

# Initialize GridSearchCV
grid_search_knn = GridSearchCV(
    estimator=knn_model,
    param_grid=param_grid_knn,
    scoring='neg_mean_squared_error',  # Use MSE as the scoring metric
    cv=5,                              # 3-fold cross-validation
    verbose=2,                         # Detailed progress output
    n_jobs=-1                          # Use all CPU cores
)

In [ ]:
# Fit GridSearchCV
grid_search_knn.fit(X_train, y_train)

In [ ]:
# Best parameters
print("Best Parameters for kNN:", grid_search_knn.best_params_)

# Best model
best_model_knn = grid_search_knn.best_estimator_

# Make predictions using the best model
y_pred_knn_tuned = best_model_knn.predict(X_test)

#### Last Model

In [89]:
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)


#### Best Model

In [90]:
# Initialize kNN Regressor with the best parameters
knn = KNeighborsRegressor(
    n_neighbors=5,        # Number of neighbors to use
    weights='distance',   # Weight points by the inverse of their distance
    metric='manhattan',   # Manhattan distance metric
    leaf_size=20,         # Leaf size for the BallTree/KDTree
    p=1                   # Power parameter for the distance metric
)

# Train the model
knn.fit(X_train, y_train)

# Predict on the test set
y_pred_knn = knn.predict(X_test)

### GPR

#### Hyperparameter Tuning

In [91]:
# Define parameter grid for GaussianProcessRegressor
param_grid_gpr = {
    'kernel': [C(1.0) * RBF(length_scale=1.0), C(1.0) * RBF(length_scale=5.0), C(2.0) * RBF(length_scale=2.0)],  # Kernels to test
    'alpha': [1e-2, 1e-3, 1e-4],  # Regularization parameter
    'n_restarts_optimizer': [0, 5, 10],  # Number of restarts of the optimizer to find optimal hyperparameters
    'normalize_y': [True, False],  # Whether to normalize the target variable
}

# Initialize GaussianProcessRegressor model
gpr_model = GaussianProcessRegressor()

# Initialize GridSearchCV
grid_search_gpr = GridSearchCV(
    estimator=gpr_model,
    param_grid=param_grid_gpr,
    scoring='neg_mean_squared_error',  # Use MSE as the scoring metric
    cv=5,                              # 3-fold cross-validation
    verbose=2,                         # Detailed progress output
    n_jobs=-1                          # Use all CPU cores
)

In [ ]:
# Fit GridSearchCV
grid_search_gpr.fit(X_train, y_train)

In [ ]:
# Best parameters
print("Best Parameters for GPR:", grid_search_gpr.best_params_)

# Best model
best_model_gpr = grid_search_gpr.best_estimator_

# Make predictions using the best model
y_pred_gpr_tuned = best_model_gpr.predict(X_test)

#### Last Model

In [94]:
gpr = GaussianProcessRegressor()
gpr.fit(X_train, y_train)
y_pred_gpr = gpr.predict(X_test)

#### Best Model

In [95]:
# Define the kernel as per the best parameters
kernel = C(1.0, (1e-3, 1e3)) * RBF(length_scale=1, length_scale_bounds=(1e-2, 1e2))

# Initialize Gaussian Process Regressor with the best parameters
gpr = GaussianProcessRegressor(
    kernel=kernel,               # Kernel function (using RBF with constant kernel)
    alpha=0.001,                 # Noise level in the data (Best parameter: 0.001)
    n_restarts_optimizer=5,      # Number of restarts of optimizer (Best parameter: 5)
    normalize_y=False,           # Normalize the output y (Best parameter: False)
    random_state=42              # For reproducibility
)

# Train the model
gpr.fit(X_train, y_train)

# Predict on the test set
y_pred_gpr = gpr.predict(X_test)

### CNN

#### Hyperparameter Tuning

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasRegressor  # Use SciKeras wrapper
from sklearn.model_selection import GridSearchCV

# Define the function to create the model
def create_model(hidden_layers=(64, 32), learning_rate=0.01):
    model = Sequential()
    model.add(Dense(hidden_layers[0], activation='relu', input_dim=X_train.shape[1]))
    for units in hidden_layers[1:]:
        model.add(Dense(units, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='linear'))
    # Specify learning rate inside the optimizer
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse')
    return model

# Wrap the model using SciKeras' KerasRegressor
keras_regressor = KerasRegressor(
    model=create_model,
    verbose=0  # Suppress training output for GridSearchCV
)

# Define the hyperparameter grid
param_grid = {
    'model__hidden_layers': [(128, 64, 32), (64, 32), (128, 64)],
    'model__learning_rate': [0.001, 0.01, 0.1],
    'batch_size': [16, 32, 64],
    'epochs': [20, 50]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=keras_regressor,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',  # Use MSE for regression
    cv=3,                              # 3-fold cross-validation
    verbose=2
)

# Fit the GridSearchCV
grid_search.fit(X_train, y_train)

In [ ]:
# Display the best parameters and score
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

Best Parameters: {'batch_size': 32, 'epochs': 50, 'model__hidden_layers': (128,
64), 'model__learning_rate': 0.01}


#### Last Model

In [ ]:
cnn = Sequential([
    Dense(64, activation='relu', input_dim=X_train.shape[1]),
    Dense(32, activation='relu'),
    Dense(y_train.shape[1], activation='linear')
])
cnn.compile(optimizer='adam', loss='mse')
cnn.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)
y_pred_cnn = cnn.predict(X_test)

#### Best Model

In [ ]:
from tensorflow.keras.optimizers import Adam

# Initialize CNN model with the best hidden layers and learning rate
def create_cnn(hidden_layers, learning_rate, input_dim, output_dim):
    model = Sequential()
    # Input layer
    model.add(Dense(hidden_layers[0], activation='relu', input_dim=input_dim))
    # Hidden layers
    for units in hidden_layers[1:]:
        model.add(Dense(units, activation='relu'))
    # Output layer
    model.add(Dense(output_dim, activation='linear'))
    # Compile the model
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse')
    return model

# Best parameters
hidden_layers = (128, 64)      # Optimal hidden layers
learning_rate = 0.01           # Optimal learning rate
batch_size = 32                # Optimal batch size
epochs = 50                    # Optimal number of epochs

# Create the model
cnn = create_cnn(
    hidden_layers=hidden_layers,
    learning_rate=learning_rate,
    input_dim=X_train.shape[1],
    output_dim=y_train.shape[1]
)

# Train the model
cnn.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

# Predict on the test set
y_pred_cnn = cnn.predict(X_test)

### XGBOOST

#### Hyperparameter Tuning

In [101]:
# Define the parameter grid for hyperparameter tuning
param_grid_xgboost = {
    'n_estimators': [50, 100, 200],         # Number of trees
    'learning_rate': [0.01, 0.1, 0.2],     # Learning rate
    'max_depth': [3, 5, 7],                # Maximum tree depth
    'subsample': [0.8, 1.0],               # Subsample ratio
    'colsample_bytree': [0.8, 1.0],        # Feature sampling ratio
    'reg_alpha': [0, 0.1, 0.5],            # L1 regularization
    'reg_lambda': [1, 1.5, 2]              # L2 regularization
}

# Initialize the XGBRegressor
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)

# Wrap the model with GridSearchCV
grid_search_xgb = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid_xgboost,
    scoring='neg_mean_squared_error',  # Minimize mean squared error
    cv=5,                              # 3-fold cross-validation
    verbose=2,                         # Output progress during the search
    n_jobs=-1                          # Use all available CPU cores
)


In [ ]:
# Perform the grid search
grid_search_xgb.fit(X_train, y_train)

In [ ]:
# Best parameters
print("Best Parameters for XGBoost:", grid_search_xgb.best_params_)

# Best model
best_model_xgb = grid_search_xgb.best_estimator_

# Make predictions using the best model
y_pred_xgb_tuned = best_model_xgb.predict(X_test)

#### Last Model

In [ ]:
xgboost = XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5)
xgboost.fit(X_train, y_train)
y_pred_xgboost = xgboost.predict(X_test)

#### Best Model

In [89]:
# Initialize XGBoost Regressor with the best parameters
xgboost = XGBRegressor(
    objective='reg:squarederror',  # Regression objective
    n_estimators=50,              # Number of boosting rounds (Best parameter: 50)
    learning_rate=0.1,            # Learning rate (Best parameter: 0.1)
    max_depth=5,                  # Maximum tree depth for base learners (Best parameter: 5)
    colsample_bytree=0.8,         # Subsample ratio of columns when constructing each tree (Best parameter: 0.8)
    subsample=1.0,                # Subsample ratio of training instances (Best parameter: 1.0)
    reg_alpha=0,                  # L1 regularization term (Best parameter: 0)
    reg_lambda=1,                 # L2 regularization term (Best parameter: 1)
    random_state=42               # For reproducibility
)

# Train the model
xgboost.fit(X_train, y_train)

# Predict on the test set
y_pred_xgboost = xgboost.predict(X_test)

### CATBOOST

#### Hyperparameter Tuning

In [90]:
# Define the parameter grid for CatBoost
param_grid_catboost = {
    'estimator__iterations': [100, 200, 300],       # Number of boosting iterations
    'estimator__learning_rate': [0.01, 0.1, 0.2],  # Learning rate
    'estimator__depth': [4, 6, 8],                 # Depth of the trees
    'estimator__l2_leaf_reg': [1, 3, 5],           # L2 regularization term
}

# Initialize CatBoostRegressor
catboost_base = CatBoostRegressor(loss_function='RMSE', verbose=0, random_state=42)

# Wrap CatBoostRegressor with MultiOutputRegressor
multi_catboost = MultiOutputRegressor(catboost_base)

# Initialize GridSearchCV
grid_search_catboost = GridSearchCV(
    estimator=multi_catboost,
    param_grid=param_grid_catboost,
    scoring='neg_mean_squared_error',  # Minimize mean squared error
    cv=5,                              # 3-fold cross-validation
    verbose=2,                         # Output progress during the search
    n_jobs=-1                          # Use all available CPU cores
)

In [ ]:
# Perform Grid Search
grid_search_catboost.fit(X_train, y_train)

In [ ]:
# Best parameters
print("Best Parameters for CatBoost:", grid_search_catboost.best_params_)

# Best model
best_model_catboost = grid_search_catboost.best_estimator_

# Make predictions using the best model
y_pred_catboost_tuned = best_model_catboost.predict(X_test)

#### Last Model

In [ ]:
catboost_base = CatBoostRegressor(iterations=100, learning_rate=0.1, depth=5, verbose=0)
multi_catboost = MultiOutputRegressor(catboost_base)

multi_catboost.fit(X_train, y_train)
y_pred_catboost = pd.DataFrame(multi_catboost.predict(X_test), columns=output_cols)

#### Best Model

In [93]:
# Initialize CatBoost Regressor with the best parameters
catboost_base = CatBoostRegressor(
    iterations=200,           # Number of iterations (Best parameter: 200)
    learning_rate=0.1,        # Learning rate (Best parameter: 0.1)
    depth=4,                  # Depth of the tree (Best parameter: 4)
    l2_leaf_reg=1,            # L2 regularization term (Best parameter: 1)
    verbose=0                 # Disable verbose output
)

# Wrap the CatBoost Regressor with MultiOutputRegressor for multi-output regression
multi_catboost = MultiOutputRegressor(catboost_base)

# Train the model
multi_catboost.fit(X_train, y_train)

# Predict on the test set
y_pred_catboost = pd.DataFrame(multi_catboost.predict(X_test), columns=output_cols)

## STEP 5: EVALUATION

#### Invert Values

In [94]:
# RF
y_pred_rf_df = pd.DataFrame(y_pred_rf, columns=output_cols)
inverted_pred_rf_df = pd.concat([X_test_merged.reset_index(drop=True), y_pred_rf_df.reset_index(drop=True)], axis=1)

inverted_pred_rf_df[normalized_df.columns] = scaler.inverse_transform(inverted_pred_rf_df[normalized_df.columns])

# MLP
y_pred_mlp_df = pd.DataFrame(y_pred_mlp, columns=output_cols)
inverted_pred_mlp_df = pd.concat([X_test_merged.reset_index(drop=True), y_pred_mlp_df.reset_index(drop=True)], axis=1)

inverted_pred_mlp_df[normalized_df.columns] = scaler.inverse_transform(inverted_pred_mlp_df[normalized_df.columns])

# SVR
y_pred_svr_df = pd.DataFrame(y_pred_svr, columns=output_cols)
inverted_pred_svr_df = pd.concat([X_test_merged.reset_index(drop=True), y_pred_svr_df.reset_index(drop=True)], axis=1)

inverted_pred_svr_df[normalized_df.columns] = scaler.inverse_transform(inverted_pred_svr_df[normalized_df.columns])

# RR
y_pred_rr_df = pd.DataFrame(y_pred_rr, columns=output_cols)
inverted_pred_rr_df = pd.concat([X_test_merged.reset_index(drop=True), y_pred_rr_df.reset_index(drop=True)], axis=1)

inverted_pred_rr_df[normalized_df.columns] = scaler.inverse_transform(inverted_pred_rr_df[normalized_df.columns])

# LASSO
y_pred_lasso_df = pd.DataFrame(y_pred_lasso, columns=output_cols)
inverted_pred_lasso_df = pd.concat([X_test_merged.reset_index(drop=True), y_pred_lasso_df.reset_index(drop=True)], axis=1)

inverted_pred_lasso_df[normalized_df.columns] = scaler.inverse_transform(inverted_pred_lasso_df[normalized_df.columns])

# KNN
y_pred_knn_df = pd.DataFrame(y_pred_knn, columns=output_cols)
inverted_pred_knn_df = pd.concat([X_test_merged.reset_index(drop=True), y_pred_knn_df.reset_index(drop=True)], axis=1)

inverted_pred_knn_df[normalized_df.columns] = scaler.inverse_transform(inverted_pred_knn_df[normalized_df.columns])

# GPR
y_pred_gpr_df = pd.DataFrame(y_pred_gpr, columns=output_cols)
inverted_pred_gpr_df = pd.concat([X_test_merged.reset_index(drop=True), y_pred_gpr_df.reset_index(drop=True)], axis=1)

inverted_pred_gpr_df[normalized_df.columns] = scaler.inverse_transform(inverted_pred_gpr_df[normalized_df.columns])

# # CNN
# y_pred_cnn_df = pd.DataFrame(y_pred_cnn, columns=output_cols)
# inverted_pred_cnn_df = pd.concat([X_test_merged.reset_index(drop=True), y_pred_cnn_df.reset_index(drop=True)], axis=1)

# inverted_pred_cnn_df[normalized_df.columns] = scaler.inverse_transform(inverted_pred_cnn_df[normalized_df.columns])

# XGBOOST
y_pred_xgboost_df = pd.DataFrame(y_pred_xgboost, columns=output_cols)
inverted_pred_xgboost_df = pd.concat([X_test_merged.reset_index(drop=True), y_pred_xgboost_df.reset_index(drop=True)], axis=1)

inverted_pred_xgboost_df[normalized_df.columns] = scaler.inverse_transform(inverted_pred_xgboost_df[normalized_df.columns])

# # CATBOOST
# y_pred_catboost_df = pd.DataFrame(y_pred_catboost, columns=output_cols)
# inverted_pred_catboost_df = pd.concat([X_test_merged.reset_index(drop=True), y_pred_catboost_df.reset_index(drop=True)], axis=1)

# inverted_pred_catboost_df[normalized_df.columns] = scaler.inverse_transform(inverted_pred_catboost_df[normalized_df.columns])

# #GRU
# y_pred_gru_df = pd.DataFrame(y_pred_gru, columns=output_cols)
# inverted_pred_gru_df = pd.concat([X_test_merged.reset_index(drop=True), y_pred_gru_df.reset_index(drop=True)], axis=1)

# inverted_pred_gru_df[normalized_df.columns] = scaler.inverse_transform(inverted_pred_gru_df[normalized_df.columns])

In [95]:
pd.set_option('display.float_format', '{:.10f}'.format)

#### MAE

In [ ]:
inverted_y_actual = inverted_actual_df[output_cols]
inverted_y_pred_rf = inverted_pred_rf_df[output_cols]
inverted_y_pred_mlp = inverted_pred_mlp_df[output_cols]
inverted_y_pred_svr = inverted_pred_svr_df[output_cols]
inverted_y_pred_rr = inverted_pred_rr_df[output_cols]

inverted_y_pred_lasso = inverted_pred_lasso_df[output_cols]
inverted_y_pred_knn = inverted_pred_knn_df[output_cols]
inverted_y_pred_gpr = inverted_pred_gpr_df[output_cols]
# inverted_y_pred_cnn = inverted_pred_cnn_df[output_cols]
inverted_y_pred_xgboost = inverted_pred_xgboost_df[output_cols]
# inverted_y_pred_catboost = inverted_pred_catboost_df[output_cols]
# inverted_y_pred_gru = inverted_pred_gru_df[output_cols]



models = {
    'RF': inverted_y_pred_rf,
    'MLP': inverted_y_pred_mlp,
    'SVR': inverted_y_pred_svr,
    'RR': inverted_y_pred_rr,
    'LASSO': inverted_y_pred_lasso,
    'KNN': inverted_y_pred_knn,
    'GPR': inverted_y_pred_gpr,
    #'CNN': inverted_y_pred_cnn,
    'XGBOOST': inverted_y_pred_xgboost
    #'CATBOOST': inverted_y_pred_catboost
}

# Prepare a dictionary to store results in wide format
mae_dict = {'Columnn (MAE)': output_cols}

# Iterate through each model and calculate MAE for each column
for model_name, inverted_y_pred in models.items():
    mae_dict[model_name] = []  # Add a column for each model
    for col in output_cols:
        # Extract actual and predicted values
        y_actual_col = inverted_y_actual[col]
        y_pred_col = inverted_y_pred[col]

        # Calculate MAE and store in the dictionary
        mae_col = mean_absolute_error(y_actual_col, y_pred_col)
        mae_dict[model_name].append(mae_col)

# Convert the dictionary to a DataFrame
metrics_df = pd.DataFrame(mae_dict)

# Display the DataFrame
print(metrics_df)

#### MSE

In [ ]:
mse_dict = {'Column (MSE)': output_cols}

# Iterate through each model and calculate MSE for each column
for model_name, inverted_y_pred in models.items():
    mse_dict[model_name] = []  # Add a column for each model
    for col in output_cols:
        # Extract actual and predicted values
        y_actual_col = inverted_y_actual[col]
        y_pred_col = inverted_y_pred[col]

        # Calculate MSE and store in the dictionary
        mse_col = mean_squared_error(y_actual_col, y_pred_col)
        mse_dict[model_name].append(mse_col)

# Convert the dictionary to a DataFrame
mse_df = pd.DataFrame(mse_dict)

# Display the DataFrame
print(mse_df)

#### RMSE

In [ ]:
rmse_dict = {'Column (RMSE)': output_cols}

# Iterate through each model and calculate MSE for each column
for model_name, inverted_y_pred in models.items():
    rmse_dict[model_name] = []  # Add a column for each model
    for col in output_cols:
        # Extract actual and predicted values
        y_actual_col = inverted_y_actual[col]
        y_pred_col = inverted_y_pred[col]

        # Calculate MSE and store in the dictionary
        rmse_col = sqrt(mean_squared_error(y_actual_col, y_pred_col))
        rmse_dict[model_name].append(rmse_col)

# Convert the dictionary to a DataFrame
rmse_df = pd.DataFrame(rmse_dict)

# Display the DataFrame
print(rmse_df)

## STEP 6: DEPLOYMENT

# SHAP ANALYSIS

In [ ]:
# Set SHAP logging level to suppress verbose outputs
shap.initjs()

## RF

In [125]:
# Initialize SHAP TreeExplainer for Random Forest
rf_explainer = shap.TreeExplainer(rf_model)

# Compute SHAP values for the test set
rf_shap_values = rf_explainer.shap_values(X_test)

In [ ]:
# Summary Plot for Random Forest
shap.summary_plot(rf_shap_values, X_test)

## MLP

In [ ]:
# Use k-means to summarize the background data into K representative samples
K = 200  # Number of representative samples to use
background_data = shap.kmeans(X_train, K)

# Initialize SHAP KernelExplainer with the summarized background data
mlp_explainer = shap.KernelExplainer(mlp.predict, background_data)

# Compute SHAP values for the test set
mlp_shap_values = mlp_explainer.shap_values(X_test)

In [ ]:
shap.summary_plot(mlp_shap_values, X_test)

## SVR

## RR

## LASSO

## KNN

## GPR

## CNN

## XGBOOST

## CATBOOST

## STEP 7: OPTIMIZATION (RR)

In [18]:
X_train.columns

Index(['ni_in', 'fe_in', 'cao_in', 'al2o3_in', 's_m', 'bc', 'loi_in',
       'mc_kilnfeed', 'fc_coal', 'gcv_coal', 'tco', 'dry_ore', 'current',
       'load', 'realisasi_beban', 'rpm', 'pry_p', 'sec_p', 'pry_v', 'sec_v',
       'total_coal', 'a_f_ratio', 'kg_tco', 'reductor_consume', 'charge_kiln',
       't_tic162', 't_tic163'],
      dtype='object')

In [19]:
y_train.columns

Index(['furnace_temp', 'ni_met', 'c_met', 'si_met', 'fe_met', 's_met',
       'ni_slag', 'fe_slag', 't_kalsin', 'pic_161', 'loi_kalsin'],
      dtype='object')

In [20]:
# RR
y_pred_rr_df = pd.DataFrame(y_pred_rr, columns=output_cols)
inverted_pred_rr_df = pd.concat([X_test_merged.reset_index(drop=True), y_pred_rr_df.reset_index(drop=True)], axis=1)

In [21]:
# Input Actual Ground Truth System
X_actual_df = inverted_actual_df[input_cols]
X_actual = X_actual_df.to_numpy()
X_actual_df

,ni_in,fe_in,cao_in,al2o3_in,s_m,bc,loi_in,mc_kilnfeed,fc_coal,gcv_coal,tco,dry_ore,current,load,realisasi_beban,rpm,pry_p,sec_p,pry_v,sec_v,total_coal,a_f_ratio,kg_tco,reductor_consume,charge_kiln,t_tic162,t_tic163
0,2.120,13.050,0.370,1.560,1.730000,0.590000,12.550,20.560,42.68,6579.85,134.779999,1344.991995,23.50,40.0,35.637500,0.550000,972.300000,340.900000,121.900000,516.400000,5000.000000,7.659600,49.154542,5.615833,2136.47,993.000000,802.000000
1,2.030,14.435,0.620,2.675,1.823258,0.561630,10.530,21.540,43.20,6656.34,134.229996,1471.976005,24.10,40.0,38.350000,0.490000,627.800000,266.300000,106.900000,458.400000,4300.000000,7.887907,53.952321,5.592917,2089.62,911.000000,798.000000
2,1.950,14.300,0.585,2.855,1.697703,0.600702,13.110,18.905,42.67,6776.36,110.510002,963.775995,24.00,35.0,29.708333,0.400000,273.100000,127.600000,70.400000,328.400000,2700.000000,8.862222,40.388930,4.604583,1558.51,814.000000,779.000000
3,2.130,13.780,0.470,1.760,1.870000,0.540000,10.350,19.765,46.83,6806.30,136.100006,1245.288011,24.80,40.0,35.329167,0.550000,1032.100000,338.800000,125.200000,516.100000,5100.000000,7.544706,51.375038,5.670833,2221.20,984.000000,811.000000
4,2.000,12.640,0.470,1.620,1.810000,0.560000,11.340,21.355,49.05,7447.71,151.869995,1439.416011,23.70,40.0,37.341667,0.590000,1002.200000,340.400000,123.300000,516.000000,5300.000000,7.237358,50.981147,6.327917,2473.90,989.000000,767.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,1.995,13.710,0.585,2.515,1.740000,0.580000,10.390,20.285,43.27,6503.57,0.000000,793.232003,23.70,15.0,20.825000,0.620588,1088.876471,334.176471,127.752941,515.988235,5294.117647,7.295827,51.250136,5.316667,2023.14,946.941176,775.941176
117,2.040,12.930,0.350,1.750,1.660000,0.610000,12.160,21.415,46.13,7264.03,143.570000,1445.663995,23.10,40.0,37.408333,0.520000,834.700000,291.700000,112.900000,480.200000,4500.000000,7.908000,49.712771,5.982083,2349.28,983.000000,784.000000
118,2.055,15.165,0.560,0.000,1.746586,0.583696,10.305,19.310,43.50,6654.38,150.410004,1285.063995,23.78,35.0,33.720833,0.620000,735.900000,311.400000,115.400000,490.700000,5000.000000,7.273200,45.024764,6.267083,2254.15,927.000000,754.000000
119,2.070,15.505,0.510,2.930,1.800052,0.566749,10.155,21.675,45.00,6747.61,150.300003,1471.976005,23.40,40.0,38.350000,0.570000,745.000000,325.800000,115.300000,503.100000,5000.000000,7.420800,49.324258,6.262500,2358.93,957.000000,817.000000


In [22]:
# Output Prediction Machine Learning (RR)
y_pred_df = inverted_pred_rr_df[output_cols]
y_pred = y_pred_df.to_numpy()
y_pred_df

,furnace_temp,ni_met,c_met,si_met,fe_met,s_met,ni_slag,fe_slag,t_kalsin,pic_161,loi_kalsin
0,0.696001,0.832609,0.490664,0.291528,0.211417,0.403603,0.652132,0.605352,0.421658,0.328102,0.387254
1,0.541318,0.776243,0.289860,0.230826,0.324862,0.588812,0.645831,0.736626,0.482648,0.311843,0.143521
2,0.651309,0.731109,0.134433,0.093683,0.472789,0.234732,0.336145,0.749649,0.456567,0.392585,0.162403
3,0.587062,0.874918,0.446204,0.315768,0.157723,0.363698,0.599056,0.580929,0.473549,0.346174,0.456673
4,0.659693,0.835690,0.422450,0.305341,0.217390,0.316461,0.652856,0.664656,0.504611,0.321500,0.452457
...,...,...,...,...,...,...,...,...,...,...,...
116,0.457901,0.827002,0.212857,0.179203,0.303711,0.224972,0.701509,0.943897,0.529491,0.318148,0.527256
117,0.682722,0.758487,0.506478,0.328901,0.295233,0.399764,0.578725,0.535616,0.471580,0.320894,0.462001
118,0.617469,0.777398,0.229000,0.135287,0.367307,0.446314,0.662704,0.976155,0.416213,0.317174,-0.081722
119,0.467147,0.849563,0.325713,0.222605,0.221215,0.718470,0.749013,0.779128,0.473366,0.309283,0.222017


In [23]:
# Output Actual Ground Truth System
y_actual_df = inverted_actual_df[output_cols]
y_actual = y_actual_df.to_numpy()
y_actual_df

,furnace_temp,ni_met,c_met,si_met,fe_met,s_met,ni_slag,fe_slag,t_kalsin,pic_161,loi_kalsin
0,1383.750000,21.640000,1.272500,1.180000,74.098500,0.552500,0.070000,8.210000,594.000000,-3.400000,0.930000
1,1348.166667,21.243333,1.161667,1.165000,74.576833,0.638000,0.068000,8.520000,614.000000,-4.800000,0.000000
2,1376.500000,21.355000,1.045000,0.995000,74.975500,0.479250,0.056667,8.933333,615.000000,-2.300000,0.000000
3,1346.400000,21.084000,1.326000,1.172000,74.524200,0.569800,0.070000,8.080000,591.000000,-3.700000,0.460000
4,1392.375000,22.103750,1.251250,1.236250,73.692625,0.487625,0.065000,8.000000,603.000000,-3.800000,0.700000
...,...,...,...,...,...,...,...,...,...,...,...
116,1361.833333,21.563333,1.145000,1.106667,74.480667,0.473333,0.063333,9.840000,626.684211,-3.257895,0.660588
117,1389.000000,21.947500,1.262500,1.225000,73.847250,0.488750,0.065000,7.810000,606.000000,-4.500000,0.600000
118,1386.666667,21.540000,1.130000,1.086667,74.542000,0.508667,0.064000,10.108000,582.000000,-3.700000,0.000000
119,1374.500000,21.173333,1.188333,1.158333,74.656833,0.614833,0.066667,9.256667,660.000000,-2.700000,0.680000


In [24]:
# Save dataframes to Excel file
with pd.ExcelWriter('input_output_hasil_RR.xlsx') as writer:
    X_actual_df.to_excel(writer, sheet_name='input_actual')
    y_pred_df.to_excel(writer, sheet_name='output_pred')
    y_actual_df.to_excel(writer, sheet_name='output_actual')

In [41]:
X_actual_df = pd.read_excel('input_output_hasil_RR.xlsx', sheet_name='input_actual', index_col=None)
y_pred_df = pd.read_excel('input_output_hasil_RR.xlsx', sheet_name='output_pred', index_col=None)
y_actual_df = pd.read_excel('input_output_hasil_RR.xlsx', sheet_name='output_actual', index_col=None)
X_actual_df = X_actual_df.iloc[:, 1:]
y_pred_df = y_pred_df.iloc[:, 1:]
y_actual_df = y_actual_df.iloc[:, 1:]
input_cols = X_actual_df.columns
output_cols = y_actual_df.columns

In [42]:
# Mendefinisikan fungsi objektif spesifik untuk sampel ke-i
def objective_function(x_scaled):
    # Mengubah bentuk input untuk prediksi
    x_scaled_reshaped = x_scaled.reshape(1, -1)
    # Melakukan prediksi menggunakan model ridge
    y_pred = ridge_model.predict(x_scaled_reshaped)[0]
    # Menghitung selisih kuadrat (MSE)
    return np.sum(y_pred - y_actual) ** 2

In [43]:
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
X_test_norm = pd.DataFrame(scaler_x.fit_transform(X_actual_df), columns=input_cols)
y_test_norm = pd.DataFrame(scaler_y.fit_transform(y_actual_df), columns=output_cols)

In [28]:
min_values = scaler_y.data_min_
max_values = scaler_y.data_max_

print(min_values.shape)
print(max_values.shape)
konstrain=[]
bounds=[(i,j) for i, j in zip(min_values, max_values)]
print(bounds)

T_furnace=1300
T_furnace_norm= (T_furnace-bounds[0][0])/(bounds[0][1]-bounds[1][0])
print("T_furnace_norm",T_furnace_norm)

ni_met=17
ni_met_norm= (ni_met-bounds[1][0])/(bounds[1][1]-bounds[1][0])
print("Ni_norm",ni_met_norm)

C_met_low=0
C_met_low_norm= (C_met_low-bounds[2][0])/(bounds[2][1]-bounds[2][0])
print("C_met_low_norm",C_met_low_norm)

Si_met_low=1
Si_met_low_norm= (Si_met_low-bounds[3][0])/(bounds[3][1]-bounds[3][0])
print("Si_met_low_norm",Si_met_low_norm)

Si_met_high=2
Si_met_high_norm= (Si_met_high-bounds[3][0])/(bounds[3][1]-bounds[3][0])
print("Si_met_high_norm",Si_met_high_norm)

fe_met_low=0
fe_met_low_norm= (fe_met_low-bounds[4][0])/(bounds[4][1]-bounds[4][0])
print("fe_met_norm",fe_met_low_norm)

s_met_low=0
s_met_low_norm= (s_met_low-bounds[5][0])/(bounds[5][1]-bounds[5][0])
print("s_met_norm",s_met_low_norm)

s_met_high=0.4
s_met_high_norm= (s_met_high-bounds[5][0])/(bounds[5][1]-bounds[5][0])
print("s_met_high_norm",s_met_high_norm)

ni_slag_low=0
ni_slag_low_norm= (ni_slag_low-bounds[6][0])/(bounds[6][1]-bounds[6][0])
print("ni_slag_low_norm",ni_slag_low_norm)

fe_slag_low=0
fe_slag_low_norm= (fe_slag_low-bounds[7][0])/(bounds[7][1]-bounds[7][0])
print("fe_slag_low_norm",fe_slag_low_norm)

T_kalsin_low=600
T_kalsin_low_norm= (T_kalsin_low-bounds[8][0])/(bounds[8][1]-bounds[8][0])
print("T_kalsin_low_norm",T_kalsin_low_norm)

pic_161_low=-1
pic_161_low_norm= (pic_161_low-bounds[9][0])/(bounds[9][1]-bounds[9][0])
print("pic_161_low_norm",pic_161_low_norm)

loi_kalsin_low=0
loi_kalsin_low_norm= (loi_kalsin_low-bounds[10][0])/(bounds[10][1]-bounds[10][0])
print("loi_kalsin_low_norm",loi_kalsin_low_norm)

loi_kalsin_high=1
loi_kalsin_high_norm= (loi_kalsin_high-bounds[10][0])/(bounds[10][1]-bounds[10][0])
print("loi_kalsin_high_norm",loi_kalsin_high_norm)

(11,)
(11,)
[(1346.4, 1405.5), (19.70875, 22.436667), (1.03, 1.326), (0.9725, 1.43625), (73.395, 75.828), (0.4395, 0.638), (0.05, 0.072), (4.99, 10.174), (492.0, 697.0), (-4.8, -1.0), (0.0, 1.04)]
T_furnace_norm -0.03348267641320444
Ni_norm -0.9929737598321345
C_met_low_norm -3.4797297297297294
Si_met_low_norm 0.059299191374663
Si_met_high_norm 2.2156334231805928
fe_met_norm -30.1664611590628
s_met_norm -2.2141057934508814
s_met_high_norm -0.1989924433249369
ni_slag_low_norm -2.272727272727274
fe_slag_low_norm -0.9625771604938274
T_kalsin_low_norm 0.526829268292683
pic_161_low_norm 1.0
loi_kalsin_low_norm 0.0
loi_kalsin_high_norm 0.9615384615384615


In [29]:
x0 = X_actual_df.iloc[0,:].to_numpy().flatten()
print(x0.shape)
min_values_input = scaler_x.data_min_
max_values_input = scaler_x.data_max_

print(min_values_input.shape)
print(max_values_input.shape)
konstrain_input=[(i,j) for i, j in zip(min_values_input, max_values_input)]
print(konstrain_input)

ni_in_exact_norm = x0[0]
ni_in_exact_norm = (ni_in_exact_norm - konstrain_input[0][0]) / (konstrain_input[0][1] - konstrain_input[0][0])
print("ni_in_exact_norm", ni_in_exact_norm)

fe_in_exact_norm = x0[1]
fe_in_exact_norm = (fe_in_exact_norm - konstrain_input[1][0]) / (konstrain_input[1][1] - konstrain_input[1][0])
print("fe_in_exact_norm", fe_in_exact_norm)

cao_in_exact_norm = x0[2]
cao_in_exact_norm = (cao_in_exact_norm - konstrain_input[2][0]) / (konstrain_input[2][1] - konstrain_input[2][0])
print("cao_in_exact_norm", cao_in_exact_norm)

al2o3_in_exact_norm = x0[3]
al2o3_in_exact_norm = (al2o3_in_exact_norm - konstrain_input[3][0]) / (konstrain_input[3][1] - konstrain_input[3][0])
print("al2o3_in_exact_norm", al2o3_in_exact_norm)

s_m_exact_norm = x0[4]
s_m_exact_norm = (s_m_exact_norm - konstrain_input[4][0]) / (konstrain_input[4][1] - konstrain_input[4][0])
print("s_m_exact_norm", s_m_exact_norm)

bc_exact_norm = x0[5]
bc_exact_norm = (bc_exact_norm - konstrain_input[5][0]) / (konstrain_input[5][1] - konstrain_input[5][0])
print("bc_exact_norm", bc_exact_norm)

loi_in_exact_norm = x0[6]
loi_in_exact_norm = (loi_in_exact_norm - konstrain_input[6][0]) / (konstrain_input[6][1] - konstrain_input[6][0])
print("loi_in_exact_norm", loi_in_exact_norm)

mc_kilnfeed_exact_norm=x0[7]
mc_kilnfeed_exact_norm= (mc_kilnfeed_exact_norm-konstrain_input[7][0])/(konstrain_input[7][1]-konstrain_input[7][0])
print("mc_kilnfeed_exact_norm",mc_kilnfeed_exact_norm)

fc_coal_exact_norm=x0[8]
fc_coal_exact_norm= (fc_coal_exact_norm-konstrain_input[8][0])/(konstrain_input[8][1]-konstrain_input[8][0])
print("fc_coal_exact_norm",fc_coal_exact_norm)

gcv_coal_exact_norm=x0[9]
gcv_coal_exact_norm= (gcv_coal_exact_norm-konstrain_input[9][0])/(konstrain_input[9][1]-konstrain_input[9][0])
print("gcv_coal_exact_norm",gcv_coal_exact_norm)

tco_exact_norm=x0[10]
tco_exact_norm= (tco_exact_norm-konstrain_input[10][0])/(konstrain_input[10][1]-konstrain_input[10][0])
print("tco_exact_norm",tco_exact_norm)

dry_ore_exact_norm = x0[11]
dry_ore_exact_norm= (dry_ore_exact_norm-konstrain_input[11][0])/(konstrain_input[11][1]-konstrain_input[11][0])
print("dry_ore_exact_norm",dry_ore_exact_norm)

current_min=0
current_min_norm= (current_min-konstrain_input[12][0])/(konstrain_input[12][1]-konstrain_input[12][0])
print("current_min_norm",current_min_norm)

load_min=0
load_min_norm= (load_min-konstrain_input[13][0])/(konstrain_input[13][1]-konstrain_input[13][0])
print("load_min_norm",load_min_norm)

realisasi_beban_min = 0
realisasi_beban_min_norm = (realisasi_beban_min-konstrain_input[14][0])/(konstrain_input[14][1]-konstrain_input[14][0])
print("realisasi_beban_min_norm",realisasi_beban_min_norm)

rpm_min=0
rpm_min_norm= (rpm_min-konstrain_input[15][0])/(konstrain_input[15][1]-konstrain_input[15][0])
print("rpm_min_norm",rpm_min_norm)

pry_p_min=0
pry_p_min_norm= (pry_p_min-konstrain_input[16][0])/(konstrain_input[16][1]-konstrain_input[16][0])
print("pry_p_min_norm",pry_p_min_norm)

sec_p_min=0
sec_p_min_norm= (sec_p_min-konstrain_input[17][0])/(konstrain_input[17][1]-konstrain_input[17][0])
print("sec_p_min_norm",sec_p_min_norm)

pry_v_min=0
pry_v_min_norm= (pry_v_min-konstrain_input[18][0])/(konstrain_input[18][1]-konstrain_input[18][0])
print("pry_v_min_norm",pry_v_min_norm)

sec_v_min=0
sec_v_min_norm= (sec_v_min-konstrain_input[19][0])/(konstrain_input[19][1]-konstrain_input[19][0])
print("sec_v_min_norm",sec_v_min_norm)

total_coal_min=0
total_coal_min_norm= (total_coal_min-konstrain_input[20][0])/(konstrain_input[20][1]-konstrain_input[20][0])
print("total_coal_norm",total_coal_min_norm)

a_f_ratio_min=0
a_f_ratio_min_norm= (a_f_ratio_min-konstrain_input[21][0])/(konstrain_input[21][1]-konstrain_input[21][0])
print("a_f_ratio_min_norm",a_f_ratio_min_norm)

kg_tco_exact_norm = x0[22]
kg_tco_exact_norm = (kg_tco_exact_norm-konstrain_input[22][0])/(konstrain_input[22][1]-konstrain_input[22][0])
print("kg_tco_exact_norm",kg_tco_exact_norm)

reductor_consume_min=0
reductor_consume_min_norm= (reductor_consume_min-konstrain_input[23][0])/(konstrain_input[23][1]-konstrain_input[23][0])
print("reductor_consume_min_norm",reductor_consume_min_norm)

charge_kiln_exact_norm=x0[24]
charge_kiln_norm = (charge_kiln_exact_norm-konstrain_input[24][0])/(konstrain_input[24][1]-konstrain_input[24][0])
print("charge_kiln_norm",charge_kiln_exact_norm)

t_tic162_min=556
t_tic162_min_norm= (t_tic162_min-konstrain_input[25][0])/(konstrain_input[25][1]-konstrain_input[25][0])
print("t_tic162_min_norm",t_tic162_min_norm)

t_tic162_max=1201
t_tic162_max_norm= (t_tic162_max-konstrain_input[25][0])/(konstrain_input[25][1]-konstrain_input[25][0])
print("t_tic162_max_norm",t_tic162_max_norm)

t_tic163_min=456
t_tic163_min_norm= (t_tic163_min-konstrain_input[26][0])/(konstrain_input[26][1]-konstrain_input[26][0])
print("t_tic163_min_norm",t_tic163_min_norm)

t_tic163_max=845
t_tic163_max_norm= (t_tic163_max-konstrain_input[26][0])/(konstrain_input[26][1]-konstrain_input[26][0])
print("t_tic163_max_norm",t_tic163_max_norm)

(27,)
(27,)
(27,)
[(1.715, 2.19), (10.97, 15.93), (0.29, 0.805), (0.0, 3.14), (1.60582, 2.21363), (0.46715, 0.63196), (9.58, 16.41), (12.505, 22.89), (36.36, 49.05), (658.97, 8485.759999999998), (0.0, 448.940002), (793.232003, 1589.544), (23.0, 33.1), (1.0, 40.0), (20.825, 38.6375), (0.4, 0.65), (273.1, 1127.1), (115.0, 342.9), (68.2, 129.2), (313.1, 519.5), (2700.0, 5600.000000000001), (6.655, 9.109231), (37.985541, 68.610635), (3.300833, 6.7225), (1369.72, 2473.9), (780.0, 1201.0), (714.0000000000001, 845.0)]
ni_in_exact_norm 0.8526315789473687
fe_in_exact_norm 0.4193548387096775
cao_in_exact_norm 0.1553398058252427
al2o3_in_exact_norm 0.4968152866242038
s_m_exact_norm 0.20430726707359195
bc_exact_norm 0.7454037983132091
loi_in_exact_norm 0.4348462664714496
mc_kilnfeed_exact_norm 0.77563793933558
fc_coal_exact_norm 0.4980299448384556
gcv_coal_exact_norm 0.7564889309665905
tco_exact_norm 0.30021828841173304
dry_ore_exact_norm 0.6928942350218039
current_min_norm -2.277227722772277
load

In [30]:
from scipy.optimize import minimize
import os
import warnings

# Jumlah sampel yang ingin dioptimalkan
num_samples = len(X_test_norm)

# Inisialisasi list untuk menyimpan hasil optimisasi
optimized_x_list = []
optimized_y_list = []

# Melakukan iterasi melalui semua sampel di X_actual_df
for i in range(num_samples):
    print(f"\nMemproses Sampel ke-{i+1}")
    # Mengambil sampel ke-i dan meratakan array-nya
    x0 = X_test_norm.iloc[i, :].to_numpy().flatten()

    # Pastikan y_actual hanya memiliki satu kolom output jika prediksi satu fitur
    y_actual = y_test_norm.iloc[i, :].to_numpy()
    
    # Mendefinisikan constraints spesifik sama dengan input user untuk sampel ke-i
    ni_in_exact_norm = x0[0]
    fe_in_exact_norm = x0[1]
    cao_in_exact_norm = x0[2]
    al2o3_in_exact_norm = x0[3]
    s_m_exact_norm = x0[4]
    bc_exact_norm = x0[5]
    loi_in_exact_norm = x0[6]
    mc_kilnfeed_exact_norm = x0[7]
    fc_coal_exact_norm = x0[8]
    gcv_coal_exact_norm = x0[9]
    tco_exact_norm = x0[10]
    dry_ore_exact_norm = x0[11]
    kg_tco_exact_norm = x0[22]
    charge_kiln_exact_norm = x0[24]

    # Mendefinisikan constraints spesifik untuk sampel ke-i
    constraints_sample = [
        # Constraints equality (tidak berubah) (harusnya equality constraint, dikunci berdasarkan data asli)
        {'type': 'eq', 'fun': lambda x, ni_in_exact_norm=ni_in_exact_norm: x[0] - ni_in_exact_norm},  
        {'type': 'eq', 'fun': lambda x, fe_in_exact_norm=fe_in_exact_norm: x[1] - fe_in_exact_norm},
        {'type': 'eq', 'fun': lambda x, cao_in_exact_norm=cao_in_exact_norm: x[2] - cao_in_exact_norm}, 
        {'type': 'eq', 'fun': lambda x, al2o3_in_exact_norm=al2o3_in_exact_norm: x[3] - al2o3_in_exact_norm}, 
        {'type': 'eq', 'fun': lambda x, s_m_exact_norm=s_m_exact_norm: x[4] - s_m_exact_norm}, 
        {'type': 'eq', 'fun': lambda x, bc_exact_norm=bc_exact_norm: x[5] - bc_exact_norm}, 
        {'type': 'eq', 'fun': lambda x, loi_in_exact_norm=loi_in_exact_norm: x[6] - loi_in_exact_norm},

        # Constraints equality (tidak berubah)
        {'type': 'eq', 'fun': lambda x, mc_kilnfeed_exact_norm=mc_kilnfeed_exact_norm: mc_kilnfeed_exact_norm - x[7]},  # mc_kilnfeed tidak berubah
        {'type': 'eq', 'fun': lambda x, fc_coal_exact_norm=fc_coal_exact_norm: fc_coal_exact_norm - x[8]},  # fc_coal tidak berubah
        {'type': 'eq', 'fun': lambda x, gcv_coal_exact_norm=gcv_coal_exact_norm: gcv_coal_exact_norm - x[9]},  # gvc_coal tidak berubah
        {'type': 'eq', 'fun': lambda x, tco_exact_norm=tco_exact_norm: tco_exact_norm - x[10]},  # tco tidak berubah
        {'type': 'eq', 'fun': lambda x, dry_ore_exact_norm=dry_ore_exact_norm: dry_ore_exact_norm - x[11]},  # dry ore tidak berubah

        # Constraints input >= min_norm
        {'type': 'ineq', 'fun': lambda x, current_min_norm=current_min_norm: x[12] - current_min_norm},  # current > current_min_norm
        {'type': 'ineq', 'fun': lambda x, load_min_norm=load_min_norm: x[13] - load_min_norm},  # load > load_min_norm
        {'type': 'ineq', 'fun': lambda x, realisasi_beban_min_norm=realisasi_beban_min_norm: x[14] - realisasi_beban_min_norm},  # realisasi_beban > realisasi_beban_min_norm
        {'type': 'ineq', 'fun': lambda x, rpm_min_norm=rpm_min_norm: x[15] - rpm_min_norm},  # rpm > rpm_min_norm
        {'type': 'ineq', 'fun': lambda x, pry_p_min_norm=pry_p_min_norm: x[16] - pry_p_min_norm},  # pry_p > pry_p_min_norm
        {'type': 'ineq', 'fun': lambda x, sec_p_min_norm=sec_p_min_norm: x[17] - sec_p_min_norm},  # sec_p > sec_p_min_norm
        {'type': 'ineq', 'fun': lambda x, pry_v_min_norm=pry_v_min_norm: x[18] - pry_v_min_norm},  # pry_v > pry_v_min_norm
        {'type': 'ineq', 'fun': lambda x, sec_v_min_norm=sec_v_min_norm: x[19] - sec_v_min_norm},  # sec_v > sec_v_min_norm
        {'type': 'ineq', 'fun': lambda x, total_coal_min_norm=total_coal_min_norm: x[20] - total_coal_min_norm},  # total_coal > total_coal_min_norm
        {'type': 'ineq', 'fun': lambda x, a_f_ratio_min_norm=a_f_ratio_min_norm: x[21] - a_f_ratio_min_norm},  # a_f_ratio > a_f_ratio_min_norm

        # Constraints equality (tidak berubah)
        {'type': 'eq', 'fun': lambda x, kg_tco_exact_norm=kg_tco_exact_norm: kg_tco_exact_norm - x[22]},  # kg_tco tidak berubah

        # Constraints input >= min_norm
        {'type': 'ineq', 'fun': lambda x, reductor_consume_min_norm=reductor_consume_min_norm: x[23] - reductor_consume_min_norm},  # reductor_consume > reductor_consume_min_norm

        # Constraints equality (tidak berubah)
        {'type': 'eq', 'fun': lambda x, charge_kiln_exact_norm=charge_kiln_exact_norm: charge_kiln_exact_norm - x[24]},  # charge_kiln tidak berubah

        # Constraints min_norm <= input <= max_norm
        {'type': 'ineq', 'fun': lambda x, t_tic162_min_norm=t_tic162_min_norm: x[25] - t_tic162_min_norm},  # t_tic162 > t_tic162_min_norm
        {'type': 'ineq', 'fun': lambda x, t_tic162_max_norm=t_tic162_max_norm: t_tic162_max_norm - x[25]},  # t_tic162 < t_tic162_max_norm
        {'type': 'ineq', 'fun': lambda x, t_tic163_min_norm=t_tic163_min_norm: x[26] - t_tic163_min_norm},  # t_tic163 > t_tic163_min_norm
        {'type': 'ineq', 'fun': lambda x, t_tic163_max_norm=t_tic163_max_norm: t_tic163_max_norm - x[26]},  # t_tic163 < t_tic163_max_norm

        # Constraints berdasarkan prediksi model Ridge
        {'type': 'ineq', 'fun': lambda x, T_furnace_norm=T_furnace_norm: ridge_model.predict(x.reshape(1, -1))[0][0] - T_furnace_norm},  # T_furnace > T_furnace_norm
        {'type': 'ineq', 'fun': lambda x, ni_met_norm=ni_met_norm: ridge_model.predict(x.reshape(1, -1))[0][1] - ni_met_norm},    # Ni_met > ni_met_norm
        {'type': 'ineq', 'fun': lambda x, C_met_low_norm=C_met_low_norm: ridge_model.predict(x.reshape(1, -1))[0][2] - C_met_low_norm},  # C_met > C_met_low_norm
        {'type': 'ineq', 'fun': lambda x, Si_met_low_norm=Si_met_low_norm: ridge_model.predict(x.reshape(1, -1))[0][3] - Si_met_low_norm},  # Si_met > Si_met_low_norm
        {'type': 'ineq', 'fun': lambda x, Si_met_high_norm=Si_met_high_norm: Si_met_high_norm - ridge_model.predict(x.reshape(1, -1))[0][3]},  # Si_met < Si_met_high_norm
        {'type': 'ineq', 'fun': lambda x, fe_met_low_norm=fe_met_low_norm: ridge_model.predict(x.reshape(1, -1))[0][4] - fe_met_low_norm},  # Fe_met > fe_met_low_norm
        {'type': 'ineq', 'fun': lambda x, s_met_low_norm=s_met_low_norm: ridge_model.predict(x.reshape(1, -1))[0][5] - s_met_low_norm},  # S_met > s_met_low_norm
        {'type': 'ineq', 'fun': lambda x, s_met_high_norm=s_met_high_norm: s_met_high_norm - ridge_model.predict(x.reshape(1, -1))[0][5]},  # S_met < s_met_high_norm
        {'type': 'ineq', 'fun': lambda x, ni_slag_low_norm=ni_slag_low_norm: ridge_model.predict(x.reshape(1, -1))[0][6] - ni_slag_low_norm},  # Ni_slag > ni_slag_low_norm
        {'type': 'ineq', 'fun': lambda x, fe_slag_low_norm=fe_slag_low_norm: ridge_model.predict(x.reshape(1, -1))[0][7] - fe_slag_low_norm},  # Fe_slag > fe_slag_low_norm
        {'type': 'ineq', 'fun': lambda x, T_kalsin_low_norm=T_kalsin_low_norm: ridge_model.predict(x.reshape(1, -1))[0][8] - T_kalsin_low_norm},    # T_kalsin > T_kalsin_low_norm
        {'type': 'ineq', 'fun': lambda x, pic_161_low_norm=pic_161_low_norm: ridge_model.predict(x.reshape(1, -1))[0][9] - pic_161_low_norm},  # pic_161 > pic_161_low_norm
        {'type': 'ineq', 'fun': lambda x, loi_kalsin_high=loi_kalsin_high: loi_kalsin_high - ridge_model.predict(x.reshape(1, -1))[0][10]},  # loi_kalsin < loi_kalsin_high
        {'type': 'ineq', 'fun': lambda x, loi_kalsin_low=loi_kalsin_low: ridge_model.predict(x.reshape(1, -1))[0][10] - loi_kalsin_low },  # loi_kalsin > loi_kalsin_low
    ]

    # Melakukan optimisasi menggunakan metode SLSQP dengan constraints yang didefinisikan
    result = minimize(
        objective_function,
        x0,
        method='SLSQP',
        constraints=constraints_sample
    )

    # Memeriksa apakah optimisasi berhasil
    if result.success:
        optimized_x = result.x
        optimized_y = ridge_model.predict(optimized_x.reshape(1, -1))[0]

        # Menyimpan input dan output yang dioptimalkan
        optimized_x_list.append(optimized_x)
        optimized_y_list.append(optimized_y)
    else:
        print(f"Optimisasi gagal untuk sampel {i+1}: {result.message}")
        # Menambahkan NaN jika optimisasi gagal
        optimized_x_list.append([np.nan] * len(input_cols))
        optimized_y_list.append([np.nan] * len(output_cols))


# Mengubah list input yang dioptimalkan menjadi DataFrame
optimized_x_df = pd.DataFrame(optimized_x_list, columns=input_cols)

# Inverse transform input yang dioptimalkan ke skala asli
optimized_x_df_original = pd.DataFrame(
    scaler_x.inverse_transform(optimized_x_df.fillna(0)),
    columns=input_cols
)

# Mengubah list output yang dioptimalkan menjadi DataFrame
optimized_y_df = pd.DataFrame(optimized_y_list, columns=output_cols)

# Inverse transform output yang dioptimalkan ke skala asli
optimized_y_df_original = pd.DataFrame(
    scaler_y.inverse_transform(optimized_y_df.fillna(0)),
    columns=output_cols
)

# Menampilkan hasil optimisasi untuk setiap sampel
for i in range(num_samples):
    print(f"\nSampel {i+1}:")
    if not optimized_x_df_original.iloc[i].isnull().any():
        print("Input yang Dioptimalkan:", optimized_x_df_original.iloc[i].values)
        print("Output yang Dioptimalkan:", optimized_y_df_original.iloc[i].values)
    else:
        print("Optimisasi tidak berhasil untuk sampel ini.")

# Fungsi untuk menghasilkan nama file unik dengan suffix numerik
def generate_unique_filename(base_name, extension):
    filename = f"{base_name}.{extension}"
    counter = 1
    while os.path.exists(filename):
        filename = f"{base_name}_{counter}.{extension}"
        counter += 1
    return filename

# Menentukan nama file dasar dan ekstensi
base_filename = 'hasil_optimisasi'
file_extension = 'xlsx'

# Menghasilkan nama file unik
unique_filename = generate_unique_filename(base_filename, file_extension)

# Mengekspor hasil yang dioptimalkan ke file Excel dengan nama unik
with pd.ExcelWriter(unique_filename) as writer:
    optimized_x_df_original.to_excel(writer, sheet_name='Input_Dioptimalkan', index=False)
    optimized_y_df_original.to_excel(writer, sheet_name='Output_Dioptimalkan', index=False)

print(f"\nHasil optimisasi telah diekspor ke file '{unique_filename}'.")


Memproses Sampel ke-1

Memproses Sampel ke-2

Memproses Sampel ke-3

Memproses Sampel ke-4

Memproses Sampel ke-5

Memproses Sampel ke-6

Memproses Sampel ke-7

Memproses Sampel ke-8

Memproses Sampel ke-9

Memproses Sampel ke-10

Memproses Sampel ke-11

Memproses Sampel ke-12

Memproses Sampel ke-13

Memproses Sampel ke-14

Memproses Sampel ke-15

Memproses Sampel ke-16

Memproses Sampel ke-17

Memproses Sampel ke-18

Memproses Sampel ke-19

Memproses Sampel ke-20

Memproses Sampel ke-21

Memproses Sampel ke-22

Memproses Sampel ke-23

Memproses Sampel ke-24

Memproses Sampel ke-25

Memproses Sampel ke-26

Memproses Sampel ke-27

Memproses Sampel ke-28

Memproses Sampel ke-29

Memproses Sampel ke-30

Memproses Sampel ke-31

Memproses Sampel ke-32

Memproses Sampel ke-33

Memproses Sampel ke-34

Memproses Sampel ke-35

Memproses Sampel ke-36

Memproses Sampel ke-37

Memproses Sampel ke-38

Memproses Sampel ke-39

Memproses Sampel ke-40

Memproses Sampel ke-41

Memproses Sampel ke-42



In [46]:
y_pred_norm_df = y_pred_df.copy()
y_pred_norm_df

,furnace_temp,ni_met,c_met,si_met,fe_met,s_met,ni_slag,fe_slag,t_kalsin,pic_161,loi_kalsin
0,0.696001,0.832609,0.490664,0.291528,0.211417,0.403603,0.652132,0.605352,0.421658,0.328102,0.387254
1,0.541318,0.776243,0.289860,0.230826,0.324862,0.588812,0.645831,0.736626,0.482648,0.311843,0.143521
2,0.651309,0.731109,0.134433,0.093683,0.472789,0.234732,0.336145,0.749649,0.456567,0.392585,0.162403
3,0.587062,0.874918,0.446204,0.315768,0.157723,0.363698,0.599056,0.580929,0.473549,0.346174,0.456673
4,0.659693,0.835690,0.422450,0.305341,0.217390,0.316461,0.652856,0.664656,0.504611,0.321500,0.452457
...,...,...,...,...,...,...,...,...,...,...,...
116,0.457901,0.827002,0.212857,0.179203,0.303711,0.224972,0.701509,0.943897,0.529491,0.318148,0.527256
117,0.682722,0.758487,0.506478,0.328901,0.295233,0.399764,0.578725,0.535616,0.471580,0.320894,0.462001
118,0.617469,0.777398,0.229000,0.135287,0.367307,0.446314,0.662704,0.976155,0.416213,0.317174,-0.081722
119,0.467147,0.849563,0.325713,0.222605,0.221215,0.718470,0.749013,0.779128,0.473366,0.309283,0.222017


In [47]:
y_pred_df = pd.DataFrame(scaler_y.inverse_transform(y_pred_norm_df), columns=output_cols)
y_pred_df

,furnace_temp,ni_met,c_met,si_met,fe_met,s_met,ni_slag,fe_slag,t_kalsin,pic_161,loi_kalsin
0,1387.533679,21.980037,1.175237,1.107696,73.909378,0.519615,0.064347,8.128144,578.439856,-3.553214,0.402744
1,1378.391868,21.826276,1.115799,1.079546,74.185390,0.556379,0.064208,8.808668,590.942769,-3.614995,0.149262
2,1384.892368,21.703156,1.069792,1.015945,74.545295,0.486094,0.057395,8.876178,585.596245,-3.308175,0.168899
3,1381.095340,22.095454,1.162076,1.118938,73.778740,0.511694,0.063179,8.001534,589.077596,-3.484540,0.474940
4,1385.387841,21.988443,1.155045,1.114102,73.923909,0.502318,0.064363,8.435575,595.445184,-3.578299,0.470556
...,...,...,...,...,...,...,...,...,...,...,...
116,1373.461929,21.964742,1.093006,1.055605,74.133928,0.484157,0.065433,9.883160,600.545631,-3.591038,0.548346
117,1386.748841,21.777840,1.179918,1.125028,74.113302,0.518853,0.062732,7.766632,588.673964,-3.580604,0.480481
118,1382.892405,21.829429,1.097784,1.035239,74.288658,0.528093,0.064579,10.050386,577.323631,-3.594739,-0.084991
119,1374.008405,22.026288,1.126411,1.075733,73.933217,0.582116,0.066478,9.029000,589.039970,-3.624723,0.230898


In [50]:
y_pred_df.to_excel('output_rr.xlsx', sheet_name='output_rr', index=False)